# MAST30034 Project 2

## Data Pre-processing

In [2]:
import pandas as pd
import requests
import numpy as np
import re
import os
import json
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from geopy import distance

### External Data

#### 1. SA2 Shapefile

In [3]:
sf = gpd.read_file("../data/raw/SA2/SA2_2021_AUST_GDA2020.shp")
# only leave useful cols
SA2_inf = ['SA2_CODE21', 'SA2_NAME21', 'geometry']
sf = sf[SA2_inf]
sf.crs = 'EPSG: 4326'
#sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

#### 2. Population

In [4]:
df_popu = pd.read_excel('../data/raw/population.xlsx', sheet_name='Table 1')
# Delete the rows that has Nan in Unnamed 1 column
df_popu = df_popu.dropna(subset=["Unnamed: 1"]) 

# Change the name of columns to the first row of the dataframe
import numpy as np
# Get the first row data
array = np.array(df_popu)
list = array.tolist()
list = list[0]

# Change the column name to the first row data
df_popu.columns = list
# Delete the first row 
df_popu.drop([6], inplace=True)

# Reset index
df_popu.reset_index(drop=True, inplace=True)


df_popu = df_popu.drop(['S/T code', 'S/T name', 'GCCSA code', 'GCCSA name', 'SA4 code', 
                  'SA4 name', 'SA3 code', 'SA3 name'], axis=1)
# Change the name of each columns 
df_popu.columns = ['SA2 code','SA2 name', '2001', '2002', '2003', '2004' , 
                  '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', 
                  '2014', '2015', '2016','2017', '2018', '2019', '2020', '2021','NaN','change 01-21', 
                  'change %', 'NaN', 'km2', 'persons/km2']

# Only keep the useful columns 
df_popu = df_popu[['SA2 code', 'SA2 name', '2016', '2017', '2018', '2019', '2020', '2021']]
df_popu['SA2_code'] = df_popu['SA2 code']

In [5]:
popu_rate = []
for i in df_popu.index:
    if df_popu['2018'][i] != 0:
        popu_rate.append(float((df_popu['2021'][i]-df_popu['2018'][i])/df_popu['2018'][i]))
    else:
        popu_rate.append('NaN')     
df_popu['popu_rate18-21'] = np.array(popu_rate).tolist()
#df_popu['popu_rate18-21'] = pd.to_numeric(popu_rate['popu_rate18-21'])

popu2022 = []
popu2025 = []
for i in df_popu.index:
    if df_popu['popu_rate18-21'][i] == 'NaN':
        popu2022.append(df_popu['2019'][i])
    else:
        popu2022.append(df_popu['2019'][i] + df_popu['2019'][i]*float(df_popu['popu_rate18-21'][i]))
df_popu['popu2022'] = np.array(popu2022).tolist()
for i in df_popu.index:
    if df_popu['popu_rate18-21'][i] == 'NaN':
        popu2025.append(df_popu['popu2022'][i])
    else:
        popu2025.append(df_popu['popu2022'][i] + df_popu['popu2022'][i]*float(df_popu['popu_rate18-21'][i]))
df_popu['popu2025'] = np.array(popu2025).tolist()
lst = ['SA2_code', 'SA2 name', 'popu2022', 'popu2025']
df_popu = df_popu[lst]

#### 3. Income

In [6]:
df_income = pd.read_excel('../data/raw/income.xlsx', sheet_name='Table 1')

df_income = df_income.dropna(subset=["Unnamed: 1"]) 
df_income = df_income.iloc[3186:19235] # the income instance counted by SA2 code
att = ['            Australian Bureau of Statistics','Unnamed: 2', 'Unnamed: 7']
df_income = df_income[att]
df_income = df_income.reset_index(drop = True)
df_income.rename(columns = {'            Australian Bureau of Statistics':'SA2_code', 'Unnamed: 2':'Year', 
                            'Unnamed: 7':'Mean Income'}, inplace = True)

df_income_2014 = df_income.loc[df_income['Year'] == 2014]
df_income_2016 = df_income.loc[df_income['Year'] == 2016]
df_income_2017 = df_income.loc[df_income['Year'] == 2017]

df_income = pd.merge(df_income_2014, df_income_2016, on='SA2_code')
df_income = pd.merge(df_income, df_income_2017, on='SA2_code')

df_income.rename(columns = {'Mean Income_x':'Mean Income2014', 'Mean Income_y': 'Mean Income2016', 
                            'Mean Income':'Mean Income2017'}, inplace = True)

#### 4. PTV

In [7]:
stops_2 = pd.read_csv("../data/raw/ptv/2/stops.txt")
stops_2 = stops_2.loc[:,["stop_name","stop_lat","stop_lon"]]
stops_3 = pd.read_csv("../data/raw/ptv/3/stops.txt")
stops_3 = stops_3.loc[:,["stop_name","stop_lat","stop_lon"]]
stops_4 = pd.read_csv("../data/raw/ptv/4/stops.txt")
stops_4 = stops_4.loc[:,["stop_name","stop_lat","stop_lon"]]

gdf_stops_2 = gpd.GeoDataFrame(
    stops_2, geometry=gpd.points_from_xy(stops_2['stop_lon'], stops_2['stop_lat']))
gdf_stops_3 = gpd.GeoDataFrame(
    stops_3, geometry=gpd.points_from_xy(stops_3['stop_lon'], stops_3['stop_lat']))
gdf_stops_4 = gpd.GeoDataFrame(
    stops_4, geometry=gpd.points_from_xy(stops_4['stop_lon'], stops_4['stop_lat']))

gdf_stops_2.crs = 'EPSG: 4326'
gdf_stops_3.crs = 'EPSG: 4326'
gdf_stops_4.crs = 'EPSG: 4326'
stops2_sa2 = gdf_stops_2.sjoin(sf, how='inner', predicate='within')
stops3_sa2 = gdf_stops_3.sjoin(sf, how='inner', predicate='within')
stops4_sa2 = gdf_stops_4.sjoin(sf, how='inner', predicate='within')

geoJSON_2 = stops2_sa2[['index_right', 'geometry']].drop_duplicates('index_right').to_json()
geoJSON_3 = stops3_sa2[['index_right', 'geometry']].drop_duplicates('index_right').to_json()
geoJSON_4 = stops4_sa2[['index_right', 'geometry']].drop_duplicates('index_right').to_json()

stops2_df = stops2_sa2.groupby('SA2_CODE21')['geometry'].count()
stops2_df = stops2_df.to_frame().reset_index()
stops2_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'stop_count'}, inplace = True)

stops3_df = stops3_sa2.groupby('SA2_CODE21')['geometry'].count()
stops3_df = stops3_df.to_frame().reset_index()
stops3_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'stop_count'}, inplace = True)

stops4_df = stops4_sa2.groupby('SA2_CODE21')['geometry'].count()
stops4_df = stops4_df.to_frame().reset_index()
stops4_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'stop_count'}, inplace = True)

stops2_sa2 = pd.merge(left=stops2_sa2, right=stops2_df, left_on='SA2_CODE21', right_on='SA2_code')

att = ['SA2_code', 'stop_count']
stops2_sa2_count = stops2_sa2[att]

stops2_sa2_count = stops2_sa2_count.drop_duplicates()

#### 5. School

In [9]:
df_school = pd.read_csv('../data/raw/school2021.csv',encoding='cp1252')

# Filter the columns and rename them.
df_school = df_school[['X', 'Y', 'Address_Line_1', 'School_Name']]  # type: ignore
df_school.rename(columns={'X':'school_X', 'Y':'school_Y',
                          'Address_Line_1':'school_addr', 'School_Name': 'school_name'}, inplace = True)

# now change our school dataframe to geodataframe by transforming the x, y coordinates to point geometry.
gdf_school = gpd.GeoDataFrame(
    df_school, geometry=gpd.points_from_xy(df_school['school_X'], df_school['school_Y']))

gdf_school.crs = 'EPSG: 4326'

# Then apply spatial join
school_sa2 = gdf_school.sjoin(sf, how='inner', predicate='within')

school_df = school_sa2.groupby('SA2_CODE21')['geometry'].count()
school_df = school_df.to_frame().reset_index()
school_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'school_count'}, inplace = True)

#### 6. Hospital

In [10]:
df_hospital = pd.read_csv('../data/raw/hospital2021.csv')
# Filter the columns and rename them.
df_hospital = df_hospital[['X','Y','addr_street','name']]
df_hospital.rename(columns={'X':'hospital_X', 'Y':'hospital_Y', 'addr_street':'hospital_addr','name':'hospital_name'}, inplace = True)

# Remove the null value.
df_hospital = df_hospital.dropna(axis=0, subset=['hospital_X', 'hospital_Y'])
# now change our hospital dataframe to geodataframe by transforming the x, y coordinates to point geometry.
gdf_hospital = gpd.GeoDataFrame(
    df_hospital, geometry=gpd.points_from_xy(df_hospital['hospital_X'], df_hospital['hospital_Y']))

gdf_hospital.crs = 'EPSG: 4326'

# Then apply spatial join
hospital_sa2 = gdf_hospital.sjoin(sf, how='inner', predicate='within')

hospital_df = hospital_sa2.groupby('SA2_CODE21')['geometry'].count()
hospital_df = hospital_df.to_frame().reset_index()
hospital_df.rename(columns = {'SA2_CODE21':'SA2_code', 'geometry':'hos_count'}, inplace = True)

### Domain Rental Prices

In [7]:
df_domain = pd.read_json('../data/raw/domain1.json')

df_domain = df_domain.T
df_domain = df_domain.drop('desc', axis = 1)
df_domain = df_domain[df_domain['rooms'].astype(bool)]
df_domain = df_domain.reset_index(drop = True)

bed = []
bath = []
parking = []
Longitude = []
Latitude = []
postcode = []

for i in df_domain.index:
    
    postcode.append(df_domain['name'][i][-4:])
    
    for r in ((',', ''), ('$', ''), ('.00', '')):
        df_domain['cost_text'][i] = df_domain['cost_text'][i].replace(*r)
    # convert room col: [bed, bath, parking] to individual col
    bed.append(df_domain['rooms'][i][0][0])
    bath.append(df_domain['rooms'][i][1][0]) 
    if len(df_domain['rooms'][i]) == 3:    # if no parking info, default no parkings      
        parking.append(df_domain['rooms'][i][2][0])
    else:     
        parking.append('0')
    Longitude.append(df_domain['coordinates'][i][0])
    Latitude.append(df_domain['coordinates'][i][1])

df_domain['Postcode'] = np.array(postcode).tolist()
df_domain['Prices'] = df_domain['cost_text'].str.extract('(\d+)') 
df_domain['Prices'] = pd.to_numeric(df_domain['Prices'])
df_domain['Bedrooms'] = np.array(bed).tolist()
df_domain['Bathrooms'] = np.array(bath).tolist()
df_domain['Parkings'] = np.array(parking).tolist()
df_domain['Longitude'] = Longitude
df_domain['Latitude'] = Latitude

df_domain = df_domain.drop('rooms', axis = 1)
df_domain = df_domain.drop('cost_text', axis = 1)
df_domain = df_domain.dropna().reset_index(drop = True)
df_domain = df_domain.drop('coordinates', axis = 1)

# remove extreme prices
df_domain = df_domain[(df_domain['Prices'] > 100) & (df_domain['Prices'] < 10000)]
# remove duplicate
df_domain = df_domain.drop_duplicates()
df_domain = df_domain.reset_index(drop = True)

In [8]:
df_domain

,name,Postcode,Prices,Bedrooms,Bathrooms,Parkings,Longitude,Latitude
0,313/394 Collins Street Melbourne VIC 3000,3000,500.0,1,2,0,-37.816680,144.961040
1,29/400 Little Collins Street Melbourne VIC 3000,3000,400.0,1,1,0,-37.815794,144.961354
2,7/562 Little Bourke Street Melbourne VIC 3000,3000,380.0,1,1,0,-37.815158,144.955702
3,232/538 Little Lonsdale Street Melbourne VIC 3000,3000,780.0,3,2,1,-37.813697,144.953207
4,1309/199 William Street Melbourne VIC 3000,3000,520.0,2,1,0,-37.814504,144.957604
...,...,...,...,...,...,...,...,...
4912,14 Inverloch Parade Inverloch VIC 3996,3996,420.0,2,1,0,-38.628714,145.735808
4913,Inverloch VIC 3996,3996,475.0,3,2,1,-38.631461,145.729364
4914,25A Veronica Street Inverloch VIC 3996,3996,400.0,3,1,1,-38.642899,145.710158
4915,16B Sandy Mount Avenue Inverloch VIC 3996,3996,400.0,2,1,1,-38.634001,145.725239


### Store Dataframe For Visualisation Only

In [9]:
# build geodataframe for visualization
gpd_domain_visual = gpd.GeoDataFrame(
    df_domain, geometry=gpd.points_from_xy(df_domain.Latitude, df_domain.Longitude))

# standardlize crs
gpd_domain_visual.crs = "EPSG:4326"

gpd_domain_visual = gpd_domain_visual.sjoin(sf, how="inner", predicate='within')
gpd_domain_visual = gpd_domain_visual.reset_index(drop = True)

gpd_domain_visual['SA2_code'] = gpd_domain_visual['SA2_CODE21']
gpd_domain_visual = gpd_domain_visual.drop('SA2_CODE21', axis = 1)

In [10]:
gpd_domain_visual.to_csv("../data/curated/visualisationMap.csv")

### Data Aggregation

In [9]:
# build geodataframe
gpd_domain = gpd.GeoDataFrame(
    df_domain, geometry=gpd.points_from_xy(df_domain.Latitude, df_domain.Longitude))

# standardlize crs
gpd_domain.crs = "EPSG:4326"

gpd_domain = gpd_domain.sjoin(sf, how="inner", predicate='within')
gpd_domain = gpd_domain.reset_index(drop = True)

gpd_domain['SA2_code'] = gpd_domain['SA2_CODE21']
gpd_domain = gpd_domain.drop('SA2_CODE21', axis = 1)

# merge population by SA2
df_popu['SA2_code'] = df_popu['SA2_code'].apply(str)
gpd_domain = pd.merge(gpd_domain, df_popu, on='SA2_code')

# merge income by SA2
df_income['SA2_code'] = df_income['SA2_code'].apply(str)
gpd_domain = pd.merge(gpd_domain, df_income, on='SA2_code')
inco_rate = []
for i in gpd_domain.index:
    inco_rate.append(float((gpd_domain['Mean Income2017'][i] - gpd_domain['Mean Income2014'][i])/
                           gpd_domain['Mean Income2014'][i]))
gpd_domain['Income rate14-17'] = np.array(inco_rate).tolist()
inco2019 = []
inco2022 = []
inco2025 = []
for i in gpd_domain.index:
    inco2019.append(gpd_domain['Mean Income2016'][i] + gpd_domain['Mean Income2016'][i]*
                    float(gpd_domain['Income rate14-17'][i]))
gpd_domain['Mean Income2019'] = np.array(inco2019).tolist()
for i in gpd_domain.index:
    inco2022.append(gpd_domain['Mean Income2019'][i] + gpd_domain['Mean Income2019'][i]*
                    float(gpd_domain['Income rate14-17'][i]))
gpd_domain['Mean Income2022'] = np.array(inco2022).tolist()
for i in gpd_domain.index:
    inco2025.append(gpd_domain['Mean Income2022'][i] + gpd_domain['Mean Income2022'][i]*
                    float(gpd_domain['Income rate14-17'][i]))
gpd_domain['Mean Income2025'] = np.array(inco2025).tolist()

In [13]:
att = ['name', 'Postcode', 'Prices', 'Bedrooms', 'Bathrooms', 'Parkings', 'geometry', 'SA2_code', 
       'Longitude', 'Latitude', 'popu2022', 'popu2025', 'Mean Income2022', 
      'Mean Income2025'] #'school_count', 'stop_count', 'hos_count'
gpd_domain = gpd_domain[att]

In [14]:
gpd_domain = gpd_domain.merge(hospital_df, on='SA2_code', how='left')
gpd_domain = gpd_domain.merge(stops2_sa2_count, on='SA2_code', how='left')
gpd_domain = gpd_domain.merge(school_df, on='SA2_code', how='left')

gpd_domain['hos_count'] = gpd_domain['hos_count'].fillna(0)
gpd_domain['stop_count'] = gpd_domain['stop_count'].fillna(0)
gpd_domain['school_count'] = gpd_domain['school_count'].fillna(0)

In [15]:
gpd_domain

,name,Postcode,Prices,Bedrooms,Bathrooms,Parkings,geometry,SA2_code,Longitude,Latitude,popu2022,popu2025,Mean Income2022,Mean Income2025,hos_count,stop_count,school_count
0,811v/162 Albert Street East Melbourne VIC 3002,3002,600.0,2,1,1,POINT (144.98553 -37.81004),206041119,-37.810043,144.985531,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0
1,PO9P/191 Powlett Street East Melbourne VIC 3002,3002,675.0,2,2,1,POINT (144.98604 -37.81011),206041119,-37.810110,144.986036,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0
2,203/33 Cliveden Close East Melbourne VIC 3002,3002,380.0,1,1,1,POINT (144.97672 -37.81710),206041119,-37.817100,144.976716,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0
3,9/322 Albert Street East Melbourne VIC 3002,3002,990.0,3,2,1,POINT (144.98027 -37.80984),206041119,-37.809840,144.980274,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0
4,107K/211 Powlett Street East Melbourne VIC 3002,3002,550.0,1,1,1,POINT (144.98599 -37.80970),206041119,-37.809699,144.985994,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4031,20 Knightsford Avenue Clyde VIC 3978,3978,500.0,4,2,2,POINT (145.34198 -38.12333),212031303,-38.123332,145.341980,21129.675792,34771.277186,71382.469390,79738.921375,0.0,0.0,5.0
4032,6 Cecil Lane Clyde VIC 3978,3978,700.0,4,2,2,POINT (145.32807 -38.12609),212031303,-38.126093,145.328066,21129.675792,34771.277186,71382.469390,79738.921375,0.0,0.0,5.0
4033,43 Bellman Avenue Clyde VIC 3978,3978,450.0,3,2,2,POINT (145.34012 -38.12376),212031303,-38.123765,145.340118,21129.675792,34771.277186,71382.469390,79738.921375,0.0,0.0,5.0
4034,24 Swanston Street Clyde North VIC 3978,3978,570.0,4,2,2,POINT (145.33160 -38.12570),212031303,-38.125702,145.331599,21129.675792,34771.277186,71382.469390,79738.921375,0.0,0.0,5.0


## By Suburb 

(below are quoted from MAST30034 subject group)

"While ABS offers data such as population in SA2 level, metrics/rankings needs to be produced with regard to Suburbs (postcodes).

In this research, we assume that: "ratio between properties counts in neighbouring areas is a good estimator of the ratio between their populations". Below is an example applying this assumption:

Given two neighbouring suburbs X, Y in the same SA2 region Z with 100,000 population. If there are 600 listed properties in X, and 400 in Y, then the population in X and Y can be reasonably estimated as 60,000 and 40,000, respectively."

In [16]:
# Group by SA2_code and then Postcode and then count, to see the property ratio of a suburb in its SA2 district.

df = gpd_domain.groupby(['SA2_code', 'Postcode']).count()

# Calculate the ratio.

pct = lambda x: x / x.sum()
out = df.groupby(['SA2_code', 'Postcode']).sum().groupby('SA2_code').apply(pct)
pct_ = out['name']

# create an array for property ratio.

array = np.empty(len(gpd_domain))

#iterate through rows of gpd_domain and assign each line with its ratio.

for i, row in gpd_domain.iterrows():
    array[i] = pct_.loc[row['SA2_code'], row['Postcode']]
ser = pd.Series(array)

# add the new column to dataframe.
gpd_domain['pct'] = ser

In [17]:
gpd_domain.head(30)

,name,Postcode,Prices,Bedrooms,Bathrooms,Parkings,geometry,SA2_code,Longitude,Latitude,popu2022,popu2025,Mean Income2022,Mean Income2025,hos_count,stop_count,school_count,pct
0,811v/162 Albert Street East Melbourne VIC 3002,3002,600.0,2,1,1,POINT (144.98553 -37.81004),206041119,-37.810043,144.985531,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000
1,PO9P/191 Powlett Street East Melbourne VIC 3002,3002,675.0,2,2,1,POINT (144.98604 -37.81011),206041119,-37.810110,144.986036,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000
2,203/33 Cliveden Close East Melbourne VIC 3002,3002,380.0,1,1,1,POINT (144.97672 -37.81710),206041119,-37.817100,144.976716,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000
3,9/322 Albert Street East Melbourne VIC 3002,3002,990.0,3,2,1,POINT (144.98027 -37.80984),206041119,-37.809840,144.980274,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000
4,107K/211 Powlett Street East Melbourne VIC 3002,3002,550.0,1,1,1,POINT (144.98599 -37.80970),206041119,-37.809699,144.985994,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000
5,1105/162 Albert Street East Melbourne VIC 3002,3002,420.0,2,1,1,POINT (144.98553 -37.81004),206041119,-37.810043,144.985531,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000
6,4/23 Albert Street East Melbourne VIC 3002,3002,475.0,1,1,0,POINT (144.97739 -37.80978),206041119,-37.809776,144.977391,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000
7,7/126 Wellington Parade East Melbourne VIC 3002,3002,1600.0,3,2,1,POINT (144.98592 -37.81621),206041119,-37.816211,144.985925,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000
8,23/2 Wellington Crescent East Melbourne VIC 3002,3002,750.0,2,2,2,POINT (144.97754 -37.81760),206041119,-37.817596,144.977542,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000
9,Unit 13P/189 Powlett St East Melbourne VIC 3002,3002,600.0,2,2,1,POINT (144.98595 -37.81041),206041119,-37.810407,144.985945,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000


In [18]:
gpd_domain.to_csv('../data/curated/gpd_domain1.csv')

## Distance 

In [19]:
# Drop unroutable points.

gpd_domain = gpd_domain.drop([2773], inplace=False)
gpd_domain = gpd_domain.drop([2813], inplace=False)
gpd_domain = gpd_domain.drop([3329], inplace=False)

gpd_domain = gpd_domain.reset_index()

In [21]:
distance = [2.640392620451656,2.127094055053294,2.317093524029894,1.993894427328646,1.863594791498904,2.11449409026853,1.49949580910743,2.796792183335552,2.317093524029894,2.17249392816665,2.2368937481776663,2.050894268021626,2.278393632190976,3.287590811618264,1.72399518166136,2.201293847674682,2.2853936126269563,2.9067918759009523,2.186693888479638,1.72399518166136,4.195388274444356,4.797186592497607,4.44698757125758,4.211088230565054,3.604589925647644,5.425284837046042,5.905583494674785,5.537184524301208,5.341285071814282,5.365185005017128,5.059685858846858,5.898883513400346,5.800283788973542,5.99548324341687,5.597084356889094,5.147085614576095,5.8459836612484395,5.514184588582988,5.811283758230083,5.199185468963888,6.093582969241104,5.945883382041926,6.19548268444487,5.392684928158478,5.24148535074131,6.170782753477912,5.042685906359478,5.166685559796838,5.042685906359478,5.122085684447595,3.9199890441488003,4.447087570978095,4.320087925925314,4.801186581318168,4.416787655662352,4.364687801274558,4.211588229167624,3.00799159306112,2.811892141133166,3.0246915463869577,3.732189569023508,3.2333909630996764,3.626289864999182,3.537190114021208,4.397587709323664,3.626289864999182,3.242790936827992,2.811892141133166,2.977791677465892,2.7137924153089323,3.3587906126242326,2.720092397701314,2.815692130512698,2.8421920564489076,2.5302929281657423,5.84698365845358,7.46897912519066,6.433282019827162,6.424782043583472,6.433282019827162,6.433282019827162,6.424782043583472,6.433282019827162,6.433282019827162,6.433282019827162,6.424782043583472,6.433282019827162,6.433282019827162,8.353276653695962,8.353276653695962,8.353276653695962,6.421282053365482,7.09898015928886,7.248079742575234,8.944675000815758,8.290476829213171,9.38247377722605,9.455473573201271,10.040671937649199,9.92847225123249,9.92847225123249,8.61747591529395,9.054074695058075,8.726375610933696,9.399873728595486,9.508673424514718,11.077169040776809,10.95796937392412,9.673772963083332,10.00197204581028,9.38577376800301,11.456867979568464,9.288074041060835,12.35396547229956,12.42596527106964,12.25446575038813,12.818564173807605,12.364865441835585,11.451767993822251,12.455365188900755,12.479965120147199,12.594064801253674,11.484567902150845,12.47876512350103,12.359765456089372,12.27146570287551,12.223065838146734,12.07046626464237,12.17196598096408,11.692667320540478,12.416665297061838,12.450165203434029,11.812766984877792,12.7349644074579,14.275160102814528,14.638859086323945,13.077663449659381,15.28025729370074,15.300557236965085,13.181363159832397,14.097360599740636,13.718761657874632,14.661459023160111,14.193860330036646,13.703661700077019,13.31976277302377,13.589062020367974,14.774058708458876,13.949861011982485,13.92226108912062,13.585762029591013,13.9099611234974,14.15016045217203,15.014058037692475,8.03797753491532,11.659267413888802,8.327276726362323,10.380870986837825,8.381576574601425,11.106768958048951,10.265971307967241,7.813578162081905,10.020771993266912,10.721270035467482,11.773267095274761,10.763669916965418,9.014574805455045,7.506579020103924,11.641667463078338,8.814775363868073,10.022571988236164,10.182371541617536,8.160777191706512,8.468876330610145,11.85246687392185,11.474267930937902,12.31546557990167,13.54796213523672,13.35166268386774,11.206068680519355,13.28196287866948,10.08147182361891,12.95046380516557,10.613470336753391,13.30146282416971,13.12346332165479,12.141566065927824,10.457670772192579,11.172368774706136,10.448370798184776,12.939963834511602,10.820669757658399,12.885463986831471,12.18896593345146,13.527062193649293,14.574259266871902,14.862758460554792,13.75646155250841,14.15696043316698,14.32195997201508,12.898063951616235,13.75026156983654,12.34316550248405,13.527062193649293,13.106063370285353,11.584067624062275,13.539062160110976,13.852561283922364,13.60026198906554,13.813861392083446,13.635361890965955,12.25096576017014,12.949363808239918,12.284565666262845,12.465165161511129,18.561348123485597,17.78805028475083,20.838741758471432,20.75104200358065,20.298043269652233,17.501051086875652,19.288746090504432,18.485348335894958,18.72524766540804,18.77124753684448,21.0792410863076,18.98644693539061,20.72114208714697,20.45964281800286,17.89914997424189,20.823341801512278,18.62094795691194,18.136849309903667,20.943241466408562,20.655842269651327,13.55246212265985,12.902963937921418,14.242860193088505,13.403362539373475,14.278460093591491,25.41092897981254,23.832533391219563,23.141035323865253,23.298034885072234,23.23273506757659,23.180735212909312,23.93963309189006,23.63243395047105,22.785136318555928,23.65813387864315,23.87543327132007,23.155135284457728,21.84513894572433,22.854336125151615,19.24314621795005,21.518639858246118,21.240840634658227,20.42774290715889,21.455540034601782,20.843541745056104,13.520362212374856,13.791761453849853,13.791761453849853,13.854861277494187,14.146560462233523,22.33033758965826,22.34723754242512,21.33074038340031,23.156835279706467,22.088238266293864,23.11833538730858,20.778441927001488,22.242537835046964,22.40513738060273,23.156435280824407,21.82543900078307,22.91223596332922,22.05203836746779,21.462540015037764,23.42913451866609,33.535006274090414,25.16172967629165,25.64212833364091,22.0685383213526,22.08713826936821,24.15733248344904,24.840730573441714,25.378829069527548,23.106635420008438,25.07382992195985,24.62313118160325,25.382829058348108,25.61222841720722,25.20072956729211,25.61222841720722,31.698411407130287,31.617611632954976,31.556011805118356,31.26381262177645,29.26901819696317,31.670211485945345,31.589311712049522,29.35881794598475,32.144310160902215,28.6185200150196,32.72910852646809,33.13540739091647,32.70090860528314,45.256973512741496,44.102276739966335,44.785674829959014,38.44159256083034,44.025576954332095,42.059882448188404,42.57258101526368,37.113096273801844,42.40818147473866,25.32712921402181,24.556331368299897,25.356329132411897,25.050829986241627,24.79843069166429,24.58583128585153,24.40433179311862,24.875930475062642,24.51343148819939,24.556331368299897,24.645031120395814,24.90473039457067,24.58583128585153,24.80253068020536,25.292729310164994,21.09544104103087,20.354243112581102,19.711344909396598,21.584539674064843,20.78964189569906,21.80103906897765,20.798641870545318,24.133732549407732,23.79983348261149,23.50713430066701,24.20383235348805,24.06863273135312,24.849730548287972,23.89973320340497,24.598831249518348,23.73693365840818,24.081532695299423,24.58303129367713,25.153029700606933,26.167826864383006,35.44760092864118,37.676494699177724,37.32359568548382,45.204073660589586,37.33209566172751,45.46707292554141,46.76286930396182,39.848288629300775,46.40337030871399,40.12078786770143,46.74136936405131,37.53589509213504,37.52659511812723,41.771283254785,37.37689553651778,46.86186902727068,45.11657390513984,45.29187341520088,32.27520979505504,31.750311262077062,31.750311262077062,31.750311262077062,38.60869209380923,39.047590867145175,37.067196402085926,38.469492482853745,38.33279286491111,38.36039278777297,31.221412740278506,31.831611034854937,32.330009641896716,30.601914471694283,29.397717837264693,31.13201299013899,32.06151039231662,32.955807892873324,33.25790704854612,32.59410890377419,32.48000922266771,32.390009474205115,33.94430513015422,30.116815827480867,34.83620263741859,38.55639223998041,30.88181368941297,33.20020720980955,29.160118501323428,4.88548634571147,4.6710869449294545,6.0225831676761645,4.96398612631496,5.113185709321848,4.22948817913963,5.274185259349388,5.024685956666958,5.538284521226862,5.080085801831714,6.08198300166148,4.844186461139188,4.354687829223158,5.142285627991422,5.701584064826224,4.884786347667872,4.812686549177277,6.055683075166298,5.14198562882988,4.466487516757811,5.584584391824845,6.041283115412282,9.81747256146195,9.81747256146195,9.532973356599621,8.494876257943785,8.971374926192995,9.532173358835509,9.93897222188646,9.96347215341239,6.148782814964832,5.409784880366372,7.617978708756521,7.721678418929539,8.37197660143208,7.503879027650045,6.844780869742272,4.921586244817025,7.5796788157996575,5.555684472596298,7.196579886510524,20.699042148913374,20.95094144488814,19.13364652398722,18.56174812236765,18.91164714444614,19.025046827509012,17.60725079006152,17.28955167798854,18.035249593861444,26.71432533699202,26.00522731882724,26.973124613682245,26.66292548064782,25.95822745018566,26.880124873604228,29.49281757147351,26.98712457455421,27.72112252312697,27.038224431736865,27.009224512787803,27.55092299881214,27.638722753423433,24.01603287836275,23.781833532918967,24.18443240770833,24.730830880596827,6.763381097243876,7.046280306577982,8.342676683321478,8.69747569170515,6.37798217438292,7.177379940171836,7.997377648386636,7.813678161802418,7.69047850612917,7.036180334806067,8.43277643150459,5.931383422567396,6.084782993835872,7.516078993552754,7.264779695901073,6.664581373376044,6.847080863314094,5.98198328114748,7.925177850175528,11.832866928701106,10.47597072104664,10.749269957211402,12.197365909974636,11.537367754582235,11.02846917688649,12.3992653456924,10.927269459726322,12.61266474926928,11.04716912462261,12.61266474926928,10.008872026525745,10.440170821102628,13.019363612599715,11.16276880153679,12.412665308241278,12.0242663937649,11.622167517578108,15.929455479277632,11.62046752232937,15.243857395433647,15.619956344286798,14.887558391242266,15.3656570550197,14.391059778890254,14.643359073747074,14.962758181068791,16.260854553061026,15.276157305159668,16.092355023994934,15.395556971453383,15.070157880900828,19.799244663728402,20.712142112300707,16.15345485322899,19.23314624589865,16.47965394154566,16.47965394154566,16.47965394154566,20.626742350981754,20.483342751764678,20.208543519792205,20.02364403656182,19.46864558770912,19.1399465063796,20.35914309888629,19.91964432722726,20.32374319782433,19.92344431660679,19.68364498681422,20.858941702015258,20.923741520908333,17.66585062628273,17.957949809904118,19.65834505752418,19.733244848189162,19.71644489514281,19.4442456559037,19.733244848189162,21.72543928026907,21.75523919698224,20.893841604474648,17.654050659262072,17.88615001057507,16.39725417184212,16.72025326910234,9.542773329209991,9.836072509477555,8.487176279464208,9.542773329209991,9.885072372529415,8.52827616459546,14.088360624894376,13.610461960557972,13.334962730541898,14.05416072047859,16.39695417268058,15.468056768826036,16.26195454998668,15.42045690186137,16.258554559489205,15.628956319133062,15.776055908009155,15.974355353788416,15.520556622095885,15.3156571947627,15.381857009742966,17.497551096657663,16.74165320929234,15.84545571404587,17.083752253170733,17.92084991359343,19.180446393187772,19.02464682862696,18.25744897284355,19.92764430486838,19.767244753163922,14.949958216843,16.62445353684993,16.99205250945939,17.111952174355682,16.78885307737495,16.773753119577332,16.295654455799898,21.40504017574221,21.56383973191845,20.90744156646455,20.250243403246543,20.772341944050137,21.92273872884319,21.058341144720178,19.09944661957143,19.101546613702222,22.07073831520391,20.354643111463158,21.28044052398177,19.75254479424836,21.38744022493175,19.54314537949205,20.169843627953288,19.18714637446221,3.2201909999918277,3.775689447447098,2.560892842643026,3.6549897847866997,4.968886112620146,3.64548981133787,3.2201909999918277,3.938188993282348,3.497190225815608,3.772389456670136,4.532587332017565,4.058788656222232,3.64548981133787,3.24449093207673,3.3290906956315744,3.61748988959395,3.984788863041872,3.4469903661175803,3.4469903661175803,3.427790419778892,1.341796249856852,1.138296818610862,1.363396189487876,2.094594145886244,1.151896780600766,2.313793533252932,2.468393101167576,2.2298937677416864,1.138296818610862,1.138296818610862,1.430896000834826,2.120094074617314,2.2778936335884064,2.0892941606990023,2.2091938255952877,1.536895704579666,1.417696037726978,2.25799368920612,2.036094309385554,2.592692753766478,3.4930902372745343,3.009591588589344,3.213391018996876,3.5873899737192363,4.120588483499884,3.6105899088784836,3.40299048909142,3.393390515922076,3.5777900005498924,2.661392561759596,3.44799036332272,4.004588807703644,4.120188484617828,3.6019899329142797,3.297390784228636,3.181191108991368,3.1432912149165624,3.2811908295053676,3.933789005579732,3.308290753764662,5.455084753759214,8.09047738818517,7.914277880639503,6.05698307153298,8.09047738818517,8.534776146428872,7.877577983210864,8.14517723530633,8.14517723530633,8.14517723530633,5.7493839312319155,8.176877146709266,8.09047738818517,8.248376946876775,5.424484839281931,7.847278067895122,8.27077688427191,7.894377936257216,6.879480772760631,5.303085178577935,4.515687379250698,5.16898555336866,4.88098635828834,4.475887490486126,4.298187987132748,4.40298769423142,6.139282841516002,6.235082573768414,4.090388567904656,6.171282752080482,5.288385219662375,4.405787686405812,4.46948750837323,4.298187987132748,4.630787057562312,6.171282752080482,4.167788351582492,4.46948750837323,10.211671459728139,10.417370884825436,26.08392709887176,26.12972697086717,26.01912727997869,27.014924496857102,26.276326561140696,25.3806290644968,24.242532245326963,24.19723237193412,24.98723016399472,27.014924496857102,24.537831420004807,26.35292634705442,23.458834435658748,23.458834435658748,12.861364054187597,13.763861531826445,12.304365610924615,15.291057263516254,13.22446303937393,13.676761775258752,13.596861998568064,12.951663801811739,14.248560177157804,13.85576127497881,12.511765031270652,14.00146086776771,12.963463768832389,14.209660285877858,13.056363509189895,14.173060388169734,14.73945880516103,12.247365770231635,14.26246013830925,37.9126940390318,33.78730556894724,37.7126945980038,38.14809338112175,52.00855464296472,26.533325842861675,26.586425694454608,27.388523452697406,26.567525747277465,26.27812655610995,26.27812655610995,40.64148641241782,38.947291147469635,41.62648365948072,39.05169085568625,33.7035058031565,2.416193247059268,2.391393316371796,3.371690576570538,1.7017952437072519,3.3493906388959163,2.486593050301124,1.522295745384622,2.4004932909385697,2.56549282978667,2.52349294717079,2.97649168109921,1.9265946154227238,2.307393551140036,2.8593920083773163,2.284093616260274,2.704292441860102,2.5008930103346265,2.56549282978667,1.9112946581840817,2.325593500273584,2.4187932397926324,2.385693332302498,3.4400903854021143,1.976094477077154,3.1002913350955423,3.501790212959252,2.638792624923432,2.4187932397926324,4.0000888202805145,2.7723922515301362,2.783592220227704,2.985291656504442,3.453890346833046,2.3499934320789997,2.4187932397926324,3.2997907775209723,2.4187932397926324,3.65448978618413,2.595192746779328,3.5077901961900926,4.0984885452662905,4.172088339564595,4.606287126036382,4.606287126036382,4.016388774724296,4.499787423688972,4.587587178300264,3.8822891495150222,2.9235918289473037,4.606287126036382,4.0099887926113995,4.066888633583866,4.021588760191023,3.6496897995994577,4.115088498871614,4.699286866114402,3.4957902297284127,4.32698790664078,3.258690892389718,4.065688636937698,5.142585627152965,5.040485912508171,3.8635892017789044,5.040485912508171,3.9893888501855157,4.757786702615093,5.079285804067602,5.075285815247042,5.636084247889554,5.211385434866595,5.052085880087795,5.052085880087795,5.442384789253936,5.052085880087795,5.052085880087795,5.052085880087795,4.933886210440245,5.045285899092842,5.899183512561888,5.523584562311305,8.750375543857055,8.881875176332965,9.24317416655005,9.00167484150874,8.759375518703315,8.72047562742337,9.9806721053408,8.719675629659259,9.488373481250376,9.13697446336418,8.46047635408697,9.42647365425221,8.13247727080105,9.817872560344005,8.400576521499083,9.181874337874966,9.323773941284331,8.967774936254491,8.352076657049794,8.273276877284761,10.521770593042051,8.72047562742337,9.321073948830454,8.824175337596389,7.237079773318694,7.9531777719194485,8.069577446597744,9.65577301339081,8.32147674257251,8.606075947155354,8.811575372811625,7.820278143356342,8.591075989078254,7.924277852690903,9.104574553917645,9.436073627421555,8.764275505008502,8.046177511997467,7.312179563424708,9.743672767722618,9.913372293434875,8.841775288406852,9.63197307990848,9.96047216179697,8.479276301543601,9.31017397929443,10.433570839548704,10.75276994742939,11.191768720485852,10.4449708076873,9.938772222445431,9.879772387342172,10.321671152293538,8.836075304337555,11.2442685737557,10.21347145469739,10.657870212661607,9.137774461128291,9.365273825297642,9.221674226639538,12.571664863858539,18.09624942337498,16.99295250694402,16.66165343288114,17.19845193260029,18.57044809805237,16.814553005547044,16.84145293036531,16.87445283813493,17.43055128391328,17.20715190828501,17.33385155417625,17.545850961665927,17.60115080711017,16.601353601411198,15.904655548590158,17.244051805154673,16.56525370230564,16.82915296474209,17.5949508244383,17.20665190968244,23.09333545718008,22.29513768803733,22.88943602705203,23.73813365505435,20.73464204941636,21.990738538792712,21.4006401880396,21.81693902453938,20.17744360671235,21.20114074561417,20.346343134660497,21.14904089122637,20.82744179005335,22.45543724002127,22.77843633728149,27.04172442195485,24.608831221569748,22.1406381198432,10.516870606736866,12.215865858269726,13.816461384816808,11.46046796950697,11.82946693820363,13.636361888171097,11.48966788789706,11.221868636360567,11.54366773697462,13.724061643061875,12.133166089404648,13.411562516455625,11.66796738957352,10.35447106062213,11.448068004163234,10.774169887619388,10.64447025011273,12.737064401588695,13.241162992699769,12.93346385267819,13.349562689736945,13.909061126012777,13.349562689736945,14.115460549153669,12.884163990464787,13.712661674923279,14.101060589399657,15.104057786155076,14.398659757649318,15.144257673801702,13.773661504436818,13.635061891804416,15.573656473688818,13.174963177719501,14.120960533781941,15.10625778000638,14.604659181908158,14.183460359103188,13.352962680234421,12.905063932052215,14.619459140544231,14.051360728304196,14.000560870283083,13.904261139428101,14.97395814976636,13.698761713771832,14.475059544122015,14.219860257370286,24.0085328993242,23.93003311872071,24.6156312025647,23.57363411480882,23.69143378557431,23.603334031801477,22.33823756757886,23.77463355304196,24.1149326019511,21.09934103013092,21.227140672947808,21.30714044935901,20.87784164919241,21.118840975631148,21.355740313528813,21.74873921514883,20.90244158043885,21.830138987647228,22.02173845215205,21.086841065066668,20.605742409673812,20.671342226330996,21.724039284181874,20.691642169595337,20.578842484855546,20.73124205891888,17.151952062561282,17.262551753449763,17.71695048346538,17.210151899900428,17.79215027329191,16.33945433338503,17.917849921978007,14.5406593607792,15.628456320530491,17.05285233953191,15.201757513097252,14.78815866905135,14.65795903294212,14.593459213210592,16.228854642496547,16.228854642496547,16.228854642496547,17.220351871392857,16.228854642496547,16.228854642496547,16.228854642496547,14.645859066759927,15.290857264075225,15.914955519803101,15.290857264075225,15.01545803377967,15.537256575421722,14.28666007067364,14.89315837559105,18.412048540758192,17.88645000973661,17.29155167239882,17.708050508339632,18.73114764891837,16.814553005547044,16.344854318292786,18.369848658701287,17.51115105864757,18.72934765394912,18.8842472210253,20.06574391889821,18.27614892057967,17.489851118178088,16.916952719353382,18.42164851392754,19.516045455232753,18.31504881185961,19.24034622577566,20.758141983737147,18.816447410516812,19.48074555389131,18.16434923304502,20.646742295084554,18.813347419180875,19.41644573360081,19.651045077926653,18.98774693175729,19.45874561537823,20.42294292057422,18.5263482213057,19.00204689179079,20.9042415754081,22.18023800916674,21.34084035517223,24.23913225482949,24.7685307752306,21.0806410823948,20.849941727169,21.914638751481558,21.1013410245412,23.56283414499331,21.62573955891661,21.829838988485687,21.46444000972753,22.109838205924888,23.035335619281955,23.172535235827162,22.609836808494887,22.69583656813693,20.04794396864672,20.413042948243334,21.95173864779225,21.319940413584803,22.254337802067617,22.768836364112147,21.517039862717894,21.96973859748477,22.535537016152983,21.812339037395738,21.564839729123587,22.340637560871198,25.1342297531503,26.21872672212463,27.28662373749364,30.17111567571997,28.53232025593654,27.819122249230688,28.17082126627843,29.42181776990857,29.95891626878926,26.42262615225268,26.821025038780455,25.99832733811178,23.756133604746868,24.39273182553899,26.17342684873179,23.496734329733552,27.524523072596445,26.67972543369417,25.023030063938734,25.20482955583319,28.00362173357902,32.50500915279621,35.00360216955902,12.184665945469359,10.064671870572559,12.18216595245651,12.086066221042554,11.848266885660262,11.60096757682914,11.618467527919089,12.330565537699284,11.319268364141202,10.16727158381992,10.169371577950717,11.675067369730014,10.159371605899317,9.989272081305002,12.217865852680006,10.992769276662992,13.672361787556136,12.477865126016406,16.106654984028438,12.48896509499346,15.837555736125266,13.145563259888386,16.64915346781689,13.783561476767705,12.04546633451387,13.62626191639918,16.615553561724184,13.59376200723213,13.67166178951254,13.151863242280767,16.584353648923816,12.13346608856619,11.496367869171495,13.074563458323446,10.826869740330265,11.9399666293716,11.782367069841536,11.262868521771306,10.991269280855281,10.48217070371851,8.902775117920392,9.60467315620816,17.78975027999957,10.657370214059036,9.916072285888756,10.83396972048676,9.511673416130138,10.04297193122102,10.200571490751084,9.76247271517925,10.60097037168914,15.947555428690665,17.51475104858607,18.908747152551232,18.733847641372247,19.011246866078082,19.535245401571444,18.217449084637952,18.841847339527366,20.798341871383776,18.08114946557737,19.76514475903313,18.223449067868792,18.040749578489713,18.62594794293764,19.5456453725049,18.93964706619006,19.41234574505974,19.21114630738557,18.32114879481097,18.21644908743281,18.078349473402977,16.076255068992182,15.407956936797119,14.995958088279442,14.391059778890254,14.416059709018754,14.1542604407131,15.273557312426304,15.70395610951856,15.096557807116525,14.902758348760392,13.75846154691869,14.922358293981135,15.10045779621657,14.699358917234916,14.173060388169734,14.415859709577726,13.983760917236733,14.106260574866381,13.902261145017821,20.053243953833963,22.72113649742697,18.49974829564897,22.041438397093312,17.90114996865217,18.81874740408863,20.977741369985893,20.794041883401675,19.389845807944088,21.709339325266317,17.73865042281692,17.45685121040847,17.72795045272192,19.31204602538419,20.38194303516348,18.375948641652638,21.012041274122193,22.063338335885877,17.923849905208847,18.299848854341487,15.9349554639059,16.74705319420009,16.352054298169797,17.29095167407574,16.19645473305001,15.603156391240448,17.32315158408125,15.70165611594674,15.70165611594674,17.72795045272192,17.412651333941277,16.735953225223042,15.19965751896646,15.915555518126183,16.59545361790087,16.91315272997385,17.4849511318729,17.77235032863014,16.85305289794493,16.740553212366684,17.77235032863014,17.688850562000948,16.16215482891371,16.761953152556682,19.67524501029104,15.940455448534168,16.82655297200872,16.04095516765074,16.773853119297847,16.385754203983012,20.20664352510244,18.649847876140488,17.885350012810957,19.425545708167583,16.24015461091463,21.66443945075553,21.03784120201481,20.13794371710932,19.65344507121899,21.480139965848227,21.812039038234193,22.05413836159859,20.57604249268115,21.23264065757608,20.836041766017555,21.0313412201814,21.24974060978397,24.867330499098436,24.9406302942352,24.867330499098436,24.87113048847797,23.438134493512347,22.997535724927662,25.73332807874968,23.990932948513738,23.06713553040541,23.56623413549078,22.561836942648167,23.85933331631732,25.3749290804275,22.23723784985972,22.803336267689478,21.9656386089437,25.44512888422833,28.47432041803842,24.25573220843481,22.9906357442122,23.205535143596784,27.53172305247345,26.791125122346767,26.23862666650692,32.25160986101374,29.54451742697924,29.264318210099017,37.47639525842921,40.67808631012595,37.089996338363115,8.327776724964892,8.532576152577565,8.833275312163162,9.024274778344902,9.01347480852939,8.717275636366923,8.71117565341557,8.98697488259318,8.645375837317355,10.476370719928696,10.793169834517048,9.058574682481204,8.59697597258858,8.508076221051633,11.390368165426656,8.987574880916265,9.27797406928892,8.50947621713883,10.435670833679499,9.160874396567024,8.68547572524347,11.066769069843351,10.55767049270658,12.826864150610264,11.258568533789203,13.544662144459759,10.019371997179714,10.733070002488134,12.649264646977402,12.340865508912225,13.51516222690813,14.282360082691538,10.141071657045254,12.42546527246707,11.20296868918342,12.74146438929131,12.322265560896621,10.236771389577152,12.05596630516784,13.174563178837445,9.588573201205405,10.714170055310989,13.805661415001298,10.966069351285755,9.821172551120968,10.555870497737326,11.20346868778599,12.459365177721315,13.217063060055896,12.095566194491385,13.585762029591013,11.978066522887435,13.032363576266535,15.121257738083482,15.04295795692102,14.989958105048599,16.86975285127077,16.072355079892134,15.666856213207865,16.358254280841663,15.388156992135348,24.248732227998833,15.086357835624096,17.01235245272374,17.56465090912256,17.305651632991296,16.675853393194128,15.963055385370335,15.130757711532313,16.78915307653649,14.918558304601603,15.273157313544248,15.16765760840198,19.77914471990509,15.095857809072925,20.52764262795238,20.09444383868573,19.015546854060183,19.621845159536566,14.908758331991232,20.397442991843153,19.34844592365129,20.543342584073077,14.309560006671346,14.645259068436841,19.797244669318122,19.13364652398722,21.522839846507708,21.81913901839069,20.32894318329106,19.93144429424791,19.93144429424791,21.278340529850976,16.09015503014363,17.076352273852695,18.66294783952782,16.73275323416659,18.4756483630051,18.08174946390045,18.48124834735388,19.65024508016254,20.400542983179083,18.468548382848603,16.59545361790087,15.748355985426777,15.494856693923785,16.981352539364398,16.098555006666803,17.730150446573226,18.868347265463576,17.45145122550071,17.389251399341003,17.12195214640708,17.859750084359373,17.25345177888299,19.16844642672609,17.25345177888299,18.229749050261173,18.70814771320015,19.2206462808344,17.38865140101792,19.176846403249268,17.73575043092201,17.803050242827933,17.390751395148712,19.51844544852509,17.65975064333137,10.734569998295845,11.072869052794704,9.82197254888508,11.660367410814455,9.925672259058098,11.486667896281638,10.786269853801581,9.907372310204035,10.89026956313614,11.787367055867236,10.692870114841506,11.486667896281638,11.159868809641885,11.61346754189339,11.64396745665016,11.099668977892458,9.834672513390359,8.824775335919472,13.025763594712611,12.743064384819533,13.024363598625415,18.290548880333684,12.625664712936098,11.6906673261302,13.025563595271583,18.41574853041721,13.04166355027434,19.467145591901406,12.8899639742546,19.11294658184082,19.076546683573724,19.33464596222036,18.9142471371795,20.199943543828,18.751547591903222,14.529059393199574,18.999546898777943,19.912144348188708,21.93093870592534,21.734839253997386,23.02413565058439,22.447537262100663,24.079032702286575,22.63143674812591,21.397640196424177,21.03304121543013,21.916138747289267,21.36644028362381,20.63534232694596,19.84144454578531,21.163840849862446,21.685039393181412,20.774841937062988,20.990141335329632,21.69433936718922,23.77683354689327,25.380229065614742,24.97733019166384,22.10743821263255,24.342731965281992,23.530934234149342,22.48193716595748,25.02983004493369,24.57913130457709,24.889530437052542,26.06652714750232,27.52712306532981,27.97532181267356,25.328529210109004,25.46642882469781,26.37002629926231,27.805722286681814,26.769625182436258,25.49992873107,25.52442866259593,27.422923356554218,25.4142289705895,24.12243258098965,27.970321826647854,26.195526786965385,28.93721912429772,26.25392662374556,26.61642561060881,28.64681993592507,25.331529201724425,24.41093177467254,26.57112573721597,27.07672432413475,24.89873041133983,26.291226519497283,28.564020167339475,24.25443221206813,24.40433179311862,25.71442813157253,28.17412125705539,24.552331379479337,34.33400404099727,34.29090416145574,33.7985055376448,31.93191075453048,32.546009038206954,22.83613617601807,32.25430985346762,22.574036908550873,27.273223774944764,26.970224621787345,27.26312380317285,27.490723167062708,27.04582441049593,30.442814916356507,30.31011528723443,29.19711839791361,28.31192087192368,28.6349199691839,27.35472354716367,26.6649254750581,28.397020634081095,27.161524087130623,30.41751498706646,28.712719751743794,27.24682384872907,29.026218875555184,27.43872331239543,28.775319576785556,27.55112299825317,28.248221049956264,27.79142232664831,28.31472086409807,28.89091925369974,28.89091925369974,28.89091925369974,28.89091925369974,28.012921707586823,28.759719620385376,29.51891749852766,28.023821677122847,29.48361759718622,29.604717258728673,29.775316781925557,30.237615489861778,29.96391625481496,29.67101707342945,29.245218263480844,29.740516879186682,29.13981855805909,28.81791945772452,29.570917353194943,29.18531843089296,29.239218280250004,29.750816850399627,29.131318581815396,30.214415554702533,28.92501915839501,30.28181536632897,29.480717605291314,29.79881671624635,34.45390370589356,33.41870659913264,33.23370711618173,33.09550750243138,34.42390378973936,33.455706495722815,31.62611160919867,34.47410364943738,33.037107665651206,34.343504014446104,32.354009574820076,35.32800126290643,35.95119952114969,34.31720408795093,36.12179904434657,35.30980131377289,32.36760953680998,33.494606387002754,32.441609329990335,32.441609329990335,33.14280737023451,36.19919882802441,37.11249627547876,35.69390024026716,35.819799888394286,36.43509816871693,36.45369811673253,36.153798954911046,36.77089723020294,34.60080329532863,35.12480183082199,32.53650906475812,33.10020748929554,33.14830735486278,33.12730741355484,33.07570755776961,37.22429596301342,36.07069918716392,36.900396868268565,36.0019993791708,37.50929516647832,36.33149845826442,36.64479758263479,36.47649805300972,38.15279336798591,37.11249627547876,41.05648525255092,41.81158314215213,42.983579866576214,40.504186796152105,40.47728687133384,41.32738449542335,42.21458201582355,40.801585964960736,40.20738762566655,41.16948493673174,40.57648659408372,39.62428925534942,8.384876565378386,6.715081232235614,8.83897529623246,7.731078392657854,6.764181095007989,7.050080295957515,6.896380725527496,7.3876793524127775,6.987280471474722,6.623181489083248,7.580778812725313,6.580281608982742,7.935977819991041,8.106077344585355,8.68097573782034,8.133577267726704,7.371079398807454,7.796978208476579,7.008880411105746,7.839278090254003,9.442473609534451,8.570876045534426,10.973769329765332,10.04447192702873,8.77347547929579,9.40197372272628,10.55697049466298,10.68347014111319,9.669072976219175,10.845569688066385,8.373876596121846,10.432370842902536,9.27247408466065,10.895069549720814,8.426076450230154,10.618570322499604,10.381870984042965,9.125674494946098,11.174268769395901,9.44917359080889,11.437168034627208,11.437168034627208,10.038371944077376,12.027766383982891,10.647370242007636,10.907669514505578,9.800272609533542,10.09547178449087,10.487370689185235,11.32396835100536,10.7149700530751,11.912166707068709,11.75346715061299,11.437168034627208,12.257865740885606,11.437168034627208,11.528367779735976,12.83466412881036,11.473567932894303,10.831369727753396,10.92446946755193,11.652067434011794,11.726267226633182,11.76276712462079,10.547170522052609,10.423170868615248,11.865066838706616,11.46296796251982,14.997558083807663,17.26175175568565,16.67475339626847,13.392262570396422,17.726950455516782,14.15016045217203,17.32385158212485,16.42445409582193,14.051560727745226,13.485062311033413,13.219363053627715,15.072857873354707,13.347162696444608,14.490159501919628,17.36885145635615,12.769364311314716,14.209260286995802,13.410362519809457,12.769364311314716,13.36346265088839,14.851758491298252,17.890849997439226,14.754258763797102,15.279157296775088,13.899261153402401,20.213643505538418,25.002330121792337,19.369745864120773,23.886833239458667,22.10613821626587,22.10613821626587,18.121149353782968,18.121149353782968,20.70314213745445,22.155038079597215,21.388240222695863,21.822339009447138,19.149846478710487,21.811539039631622,18.23684903041767,19.970144186086827,18.32324878894176,22.111438201453108,22.260137785857427,20.68084219977983,26.3085264711462,24.893530425873102,24.363331907707877,26.25542661955327,25.43122892307688,27.74822244738626,25.298429294234293,26.202126768519307,28.552420199759847,27.31842364861709,22.56073694572251,23.36723469166792,22.7099365287294,25.32462922100896,22.82443620871793,24.19173238730585,23.836133381158067,25.03943001810303,23.36723469166792,24.92323034286576,19.324645990168957,24.320332027886856,20.541342589662797,20.64324230486656,25.418028959969032,26.71052534761248,20.39014301224563,18.04974955333597,20.01744405388995,18.22014907709183,24.598731249797833,25.676028238895153,18.064249512810502,26.244926648899302,18.38124862683988,19.225246267978044,26.126326980369697,19.936144281112067,19.293046078486533,31.945710715961415,34.50370356670953,34.38730389203124,32.89430806475722,34.51790352702252,35.305301326349756,32.297709732170695,33.89460526905875,34.43080377045483,33.63580599236853,32.61890883446166,33.97660503988023,33.30950690433134,30.49831476124178,32.96380787051445,30.448014901823232,31.596011693323955,31.251812655314772,33.18090726375034,34.367403947648945,35.03840207229789,34.48160362847594,34.17700447979029,34.43470375955487,34.12050463769988,35.23790151472332,33.81160550103213,32.245909876944445,34.847002607234096,38.85839139593269,38.33169286798546,35.27220141885962,34.5627034018128,38.08859354741592,33.94840511869529,34.88510250074993,36.89889687246086,36.401598262344734,34.36300395994633,38.14669338503456,36.57379778106985,36.20879880119375,36.42589819442964,36.80309714020844,36.4291981852066,38.376692742216754,33.39610666229646,34.69030304518866,37.10389629951456,34.023404909080796,37.922294012201135,37.74469450856827,37.85979418687989,36.743997305384674,37.86429417430302,35.927999585990435,38.4705924797794,36.276598611702234,38.39049270364768,38.23779313042281,36.04789925088672,35.31600129644475,36.81859709688811,35.915499620926184,36.700697426402115,36.85929698313731,37.73079454741683,36.87899692807857,36.42939818464763,37.500095192191026,36.19529883892436,37.50949516591935,38.2741930286899,39.35779000017961,40.48808684114935,38.738791730197946,36.74889729168986,37.676494699177724,38.89409129615619,36.95869670532824,41.85788301275012,43.75567770866481,44.26247629222976,51.445856215632446,10.459670766602859,12.212865866654306,12.687064541331695,12.687064541331695,12.687064541331695,10.071571851288025,12.470565146418885,10.75866993093972,9.783872655369246,10.23917138286949,9.61247313440825,10.239571381751546,11.301568413610225,10.506270636362382,10.45897076855926,12.069166268275689,12.587064820817695,11.27266849438168,10.373871006401846,12.871464025959511,12.489865092478086,12.489865092478086,9.196174297908469,12.913363908854876,12.271765702037051,14.348059899069234,13.314662787277557,13.30016282780303,12.87296402176722,12.414365303490015,13.616661943229838,12.303065614557935,13.804861417237186,14.452759606447392,14.065360689176156,14.229860229421684,12.657864622941606,13.619661934845258,11.77396709331836,13.45776238733309,12.235165804328929,15.28945726798803,15.494856693923785,16.747753192243692,15.179357575702115,15.688456152838889,16.109354976482315,15.478856738641547,15.69545613327487,16.30195443819228,15.288657270223919,16.067855092469006,16.56985368944929,16.4449540385273,20.44344286327959,15.928755481234031,15.41295692282282,14.020760813826913,14.81365859778242,16.038555174358404,15.20295750974342,16.700853323322626,16.35875427944423,16.90535275177376,16.194854737521784,16.068355091071577,19.373445853779792,19.41214574561871,19.82044460447737,22.1299381497482,19.75264479396888,21.049941168197,23.86373330401993,21.71593930682024,19.78974469027957,19.43824567267286,18.90014717658703,17.98144974422491,21.08704106450769,21.419640134937257,19.522045438463593,19.899944382286,18.327348777482836,21.45694003068898,20.471742784185054,21.15744086774955,17.952049826393793,22.30513766008873,20.75094200386014,18.412048540758192,20.71724209804692,22.729036475347574,22.729036475347574,22.117838183566008,19.5206454423764,19.952444235555852,18.016349646684297,19.79604467267195,20.270343347069858,20.550342564509055,20.59234244712494,20.736342044665097,19.43434568357282,21.104341016156617,22.64333671486708,19.920544324711884,22.228737873616033,22.64333671486708,17.943349850709076,17.78835028391238,18.269648938746258,18.026549618176723,17.605550794812782,18.689847764346087,19.78234471096154,19.02664682303724,19.600245219905542,24.596631255667038,22.37973745159217,24.4542316536551,23.50373431016953,25.37512907986853,24.93683030485567,24.7685307752306,25.51012870256243,25.01333009104888,24.57393131911036,24.87683047254727,27.07442433056293,24.96863021597912,28.114921422511102,28.993318967506077,26.229626691660663,25.015030086297614,29.050618807360596,25.156529690824925,25.199829569807488,25.26462938870056,25.11032981994746,25.157329688589037,25.857927730510124,25.24242945074645,26.7206253193844,28.06102157315405,27.122324196689135,27.895522035703383,26.790125125141632,29.886416471416613,27.41362338254642,28.644719941794275,30.778213978960462,30.505514741118784,27.44012330848263,30.65671431853595,27.395123434251328,30.375515104450585,28.88191927885348,27.911321991544597,26.361426323298108,30.113015838101337,35.90679964524147,27.276023767119153,27.992021765999397,29.11401863016647,29.07671873441475,29.118118618707552,29.163518491820902,28.118521412449603,35.97929944261412,27.43792331463132,28.502720338664393,27.285023741965414,35.67710028722081,30.32791523748592,28.69721979506412,36.250198685486545,27.46322324392136,31.503911950730558,31.211912766829684,32.14211016705091,31.663511504670904,32.595508899861386,32.95190790377328,33.73180572406197,30.860613748664,32.63150879924642,31.611311650562595,32.763308430883875,31.27611258739967,32.642508768502964,39.655089169267725,31.36911232747769,32.35550957062778,31.039413248943028,40.62578645629713,38.92719120364632,31.72341133725879,32.3356096262455,37.44329535093908,40.28068742080331,29.967516244753465,36.9876966242773,39.37938993981063,37.32999566759671,36.50589797084084,37.339495641045545,38.529192316000604,35.30610132411387,34.23160432719094,34.25010427548603,35.051102036803165,37.741194518350284,29.89781643955521,28.545520219044384,33.347006799524095,32.70800858543963,31.12041302255937,34.11440465474853,33.63940598230703,33.4735064459743,32.31670967906835,32.99990776962,33.01550772602018,32.221209945977485,33.777005597734295,37.6198948573668,39.316990114209894,36.25229867961734,35.036202078446586,36.4298981832502,35.667000315448895,35.301501336970226,34.78010279421022,36.59469772265728,37.098296315165776,36.48959801639705,35.23830151360538,35.55650062428093,35.100901897619146,35.51530073942916,37.825994281346155,36.13379901080825,37.78339440040719,36.29439856195373,36.049199247253405,37.581494964689426,38.601592113652735,37.55929502673531,37.576894977545784,37.00339658039799,35.03480208235939,34.04400485150667,34.48080363071183,37.125596238866095,34.914502418581044,35.72980013993168,33.88610529281507,33.64170597587886,33.66330591550988,34.92170239845805,34.80420272685411,34.30260412875588,7.038680327818918,7.8649780184261,8.723875617920847,6.231782582991452,7.951377776950196,6.64548142675787,7.692878499421506,7.064880254593586,7.315579553922184,7.263079700652335,6.816980947439379,6.569281639726202,6.252782524299392,8.446576392935524,7.058780271642233,7.594378774715215,7.172179954705109,6.9013807115531955,6.662381379524735,9.008074823621635,8.542576124628965,9.29877401115583,8.346476672701009,8.630175879799229,9.303373998299476,8.521576183321024,9.441573612049826,8.526776168787752,8.502076237820795,8.51097621294654,8.464676342348557,8.695375697574356,8.594575979296245,8.732875592767105,8.386376561186095,7.900577918929084,9.104974552799701,8.727875606741407,10.333871118196246,9.768072699528036,8.878375186114976,9.418573676331604,9.384573771356845,8.221577021779025,9.507373428148036,10.030771965318312,9.915372287845155,8.94667499522604,10.031671962802939,9.346973876443581,8.417176475104409,8.8456752775069,9.508573424794205,10.272071290918594,8.798775408585831,9.817772560623492,9.12867448656152,8.190377108978655,9.757072730271494,10.649570235858944,9.78897264111546,12.042366343177935,10.988669288121919,11.53146777107191,9.733772795391731,12.93546384708847,13.58346203601919,9.622773105621192,10.308871188067744,11.390668164588199,11.658167416963149,9.94097221629674,10.632770282812592,13.855561275537784,10.078771831165032,10.957369375601036,13.300262827523541,12.386865380348667,11.22776861987089,12.06526627917564,13.9049611374717,11.185668737534499,14.814658594987561,12.47576513188561,12.089666210981058,10.873769609251331,13.95896098654926,14.814658594987561,13.409262522883802,12.087366217409237,11.185668737534499,13.660661820256,12.985163708183928,12.303365613719476,16.39295418386002,16.906252749258382,15.386056998004555,17.032052397664994,14.01266083646528,17.94304985154753,17.82115019224097,17.69815053600875,15.32045718134737,14.395959765195439,15.992255303760421,16.506253867202382,17.02415241974439,14.673858988503847,18.42114851532497,18.04344957094359,18.213049096935332,18.018849639697148,17.849050114264372,15.201957512538279,15.603556390122504,19.604545207887643,19.77654472717172,17.381351421420398,16.77195312460808,19.733544847350704,18.6163479697683,19.14904648094637,19.55944533393583,16.05575512628681,18.08714944880821,17.950149831704028,20.161943650032683,16.14415487922119,17.19085195384123,16.344454319410733,17.216251882851783,17.191751951325852,19.058546733881204,18.0449495667513,18.15164926853974,16.38265421264708,18.80674743762695,15.860855671005027,17.932349881452538,17.407151349313008,18.0449495667513,20.77954192392714,19.23764623332178,18.57614808212167,16.525353813820555,16.601253601690683,18.679947792015202,18.965046995200613,16.058855117622745,20.1156437794347,20.04174398597485,20.02764402538238,19.44904564248837,21.481039963332854,20.111543790893624,20.786741903804153,20.054743949641672,20.76694195914238,21.272540546061165,20.686742183290153,20.89414160363619,19.74204482359439,20.196543553330525,22.38383744013325,24.83463059049036,24.83463059049036,24.89583041944493,25.099329850690918,21.494939924484303,21.35414031800059,21.3749402598675,21.431540101678422,21.46664000357884,21.54943977216443,21.61783958099601,21.510339881443457,21.7349392537179,22.889536026772543,19.97964415953566,21.998438517272287,23.59893404409886,23.911133171543568,21.998438517272287,21.784539115092844,26.440226103063143,21.784539115092844,26.060627163992,21.2092407229758,22.561636943207137,21.796539081554524,26.307226474779522,20.96414140799599,27.649522723238942,28.69121981183328,26.90762479674558,27.06462435795256,27.003424528997993,27.0735243330783,23.762133587977708,23.762133587977708,28.430520540453283,25.279929345939202,28.98071900272131,27.181124032351367,27.649522723238942,26.46792602564552,23.56313414415485,19.67664500637824,18.30504883980821,18.83094736999134,20.92514151699553,18.54824816009826,17.60915078475129,17.36555146557918,17.54105097508125,18.928647096933517,18.649747876419973,16.81015301784443,21.35894030458526,21.059841140527887,18.247649000233178,18.070349495761857,17.395551381733384,18.517548245900464,18.517548245900464,21.059841140527887,21.337440364674748,20.26074337390051,20.75774198485509,20.28284331213411,19.500845497714625,21.52543983924107,21.924538723812443,22.118838180771146,23.19273517937099,22.294437689993728,23.90313319390245,21.942538673504963,23.16583525455273,23.333534785854702,24.16333246667988,24.39683181408007,23.45343445075099,23.363534702008902,25.76812798148855,30.00321614497696,26.25032663380706,24.373531879200304,24.458531641637205,25.664028272433473,26.33762638981578,33.142207371911425,33.281606982307935,25.73332807874968,30.161615702271142,24.91303037137333,25.19242959048945,30.1663156891353,27.217023932015895,32.92670797420375,25.278029351249433,26.19052680093968,24.24373224197313,28.32722082916232,27.41412338114899,34.661803124842166,26.69052540350968,34.39080388224923,34.16000452730292,34.01370493619093,28.780219563090743,34.91780240935801,27.422323358231136,33.68150586464342,34.50510356279673,27.42482335124399,33.57480616285498,27.159224093558805,27.153824108651047,33.41240661674025,33.24470708543828,29.04941881071443,34.910702429201514,31.02521328863004,46.97096872235145,45.963471538172904,46.65376960888105,47.22116802307749,46.795769212010924,48.18116534001189,49.07746283497887,49.39606194453646,48.948163196354265,47.66676677768787,49.56826146326158,47.24866794621884,47.40526750854376,49.53636155241761,47.69406670138819,49.250162352306546,47.290967827996255,47.653066815977446,46.321970536215595,46.043971313186674,46.77526926930556,46.607769737444606,47.407967500997636,47.1662681765153,48.97636311753921,48.746563759798036,46.86086903006554,48.18586532687604,48.53546434979298,47.00726862089804,48.62046411222988,46.83336910692419,50.48595889841855,53.032451781307564,53.17045139561689,52.51535322652967,50.422459075892164,52.72565263877061,51.06355728410741,52.726952635137295,50.845357893945874,50.391059163650766,52.65195284475179,50.41585909433824,52.3276537511249,52.466453363198326,50.499558860408456,50.864957839166614,52.65335284083899,51.31695657588989,51.157057022788,51.218956849786174,55.34454531931175,53.629250113335125,52.71365267230893,54.9076465403861,55.13234591238105,55.72414425838291,54.16504861584912,54.24974837912449,54.238948409308975,55.42704508873581,56.788141284651864,56.04144337157382,54.271248319034996,53.79654964575503,53.285651073649014,55.57514467481705,51.27475669383299,51.090057210043625,50.869857825471804,52.74335258930159,50.77095810188345,53.11935153843424,50.44085902446674,53.05415172065911,51.762755329941314,50.49685886795458,50.93115765414689,52.915052109424124,51.31295658706934,50.36185924526068,51.444656218986275,56.007543466319575,59.566033520810265,59.47393377821687,56.34794251494923,50.62085852139194,49.26306231625285,49.41956187885725,46.957668759523095,49.40676191463147,47.65976679725188,51.65405563374259,49.56796146410003,48.58806420278334,51.4641561644865,50.75575814436532,47.322767739119705,50.04526013011336,52.12965430450718,51.939854834971605,49.81536077265167,48.10446555437765,47.838366298089895,47.718166634032066,5.537384523742236,4.837386480144236,5.703784058677533,5.189885494956085,5.438284800712863,5.778683849342517,5.414884866112586,5.819983733914801,5.654684195905158,5.789083820275974,5.920983451633941,5.873083585507734,5.43848480015389,4.884786347667872,5.855383634976755,5.896283520666982,5.97648329651921,4.227188185567808,6.202682664321878,5.727383992718836,51.28585666281004,61.10822921057718,63.40282279749142,64.80091888999765,63.68062202107931,63.52172246518257,61.086229272064095,64.01372109011145,63.455622649922816,61.98542675892598,63.70282195903342,62.811824449253685,62.412925564123334,62.52572524886313,63.89152143164333,60.06153213595714,62.811824449253685,61.85072713539363,61.30792865244363,60.98182956384748,59.4548338315987,66.96381284499496,75.4775890501159,74.31399230221498,74.95259051741739,75.3591893810273,75.15678994670698,74.74069110964821,76.94678494390757,76.3518866065698,77.82118250008199,77.62278305458221,76.96058490533852,76.6619857398837,76.6619857398837,77.37668374239726,76.78528539527747,77.19108426112328,76.40188646682681,77.11878446319167,75.44068915324621,76.10978728320539,76.57218599086212,76.90778505290713,75.54258886845,78.8797795414432,77.45828351433669,71.94279892938701,71.34060061245171,71.32680065102079,72.02509869937005,70.71440236259305,71.08950131424106,70.95710168428053,71.13180119601847,71.3704005291649,75.91468782848258,70.92790176589044,71.17510107500105,74.37579212949264,74.23059253530631,88.45905276864082,86.88065718004782,86.80225739916484,85.02136237653103,80.06107623987509,87.8382545036899,86.49575825578944,86.49495825802533,82.57316921890727,83.69246609062048,85.79886020352738,83.76506588771365,84.45546395814229,84.05286508335294,81.6689717460197,81.92407103305091,80.88567393523353,82.75346871499401,82.75076872254014,83.17456753807848,81.97737088408488,80.8447740495433,81.62417187122941,81.41957244305777,82.98156807748646,73.29359515409014,81.09887333936938,79.72607717615318,80.93117380806738,78.82037970745789,82.96066813589903,83.8183657387476,84.34136427703584,83.37156698749106,82.40746968201557,82.08847057357592,82.06737063254747,83.90146550649473,86.06205946792024,86.81085737512906,77.65408296710311,77.68708287487273,78.60358031338355,77.72598276615267,78.77197984272911,79.21937859230876,77.03238470466756,78.30398115072359,78.11378168230597,79.2690784534042,78.4312807949379,78.63358022953774,78.69768005038719,78.3112811303211,77.69408285530871,97.01452885721608,99.29952247096098,97.32082800115046,100.8903180248977,101.07081752042546,97.8218266009256,97.96662619622987,102.14111452908679,101.72531569118959,97.12272855481221,97.0397287867856,97.23252824793659,98.95112344469018,97.89002641031615,101.73961565122308,99.15522287425927,100.0418203963364,99.22152268896005,99.72702127615833,111.78208758384154,112.74728488624267,112.30458612352719,108.95489548546973,113.17708368501182,113.78558198433953,112.58498533984844,108.66509629542016,111.38168870290349,108.33999720402915,107.94549830660142,113.10898387534182,107.58939930185105,112.099686696194,115.83807624788936,113.31288330546985,117.16317254442039,114.49717999551716,113.8646817632661,112.28398618110131,91.22654503386575,91.0303455822173,91.56364409171844,91.54434414565925,91.40234454252938,91.16104521692908,89.54654972923056,90.92844586701351,91.02834558780701,91.43094446259636,91.203245098986,91.64874385387586,91.63424389440134,108.80779589659365,110.34629159670153,108.63569637758906,108.03239806372808,109.54649383203056,110.6382907806024,101.11351740108493,102.98151218028646,102.81791263752555,101.19011718699865,105.61650481583035,103.88170966435348,104.33570839548705,102.80941266128187,102.87091248939797,102.98651216631215,103.0441120053282,102.00211491757234,104.9954065517179,100.22451988571548,100.22451988571548,105.3275056235449,101.64461591673478,102.10001464395555,103.7118101392002,101.91961514814828,113.68428226745884,112.85068459725414,113.78208199412153,113.50838275907472,199.66684195757043,199.9267412311863,199.742441746279,200.38563994862508,202.46183414593673,195.2367543390797,200.349440049799,200.00104102352822,191.13966578990065,114.20148082195726,112.72598494577319,114.64367958607015,114.56717979987694,115.2137779927205,132.55262953292242,124.59005178727465,137.51641565979634,139.00281150551646,151.2545772634507,152.47337385707533,152.30797431934516,154.85716719468806,153.69807043421028,150.85057839257416,197.07164921079112,197.78424721917386,195.78745279995033,253.90269037554347,216.95279364562046,234.84474363998535,269.3340472469408,248.49290549517713,255.02098725005158,257.2489810231035,257.4660804163393,257.43038051611586,255.92778471567252,259.02597605663726,261.41166938893974,254.2012895409983,257.2195811052723,255.6548854783898,260.0471732025262,259.9003736128117,260.93747071426236,260.97227061700124,260.5906716835198,256.65528268241184,255.70148534814933,257.01448167849816,257.638579934226,259.0456760015785,288.72829304258755,288.8181927913297,313.68382329517937,274.0042341943857,335.67496183285414,343.8740389175175,344.4018374423904,308.7283371451081,312.49662661323714,308.3155382988263,309.3595353809925,311.21573019317333,309.4981349936249,310.63973180301264,311.3741297504675,311.3225298946823,310.524032126378,310.66373173593604,311.1546303639393,310.6642317345386,363.49998406559513,363.64898364916104,363.73078342054146,364.8944801681629,358.8969969303357,364.10338237917665,360.3681928185377,357.70140027187034,358.9522967757799,360.1605933987506,360.0958935795781,359.8247943372646,359.27089588533755,361.8116887841573,363.96678276095446,359.81399436744914,363.724283438708,367.13357391019184,376.70504715918935,352.09501594097344,414.26644218005544,99.57412170349241,125.25354993288505,128.28364146417977,98.18122559645292,87.45545557356229,49.28676225001467,48.685463930563984,46.759569313184855,48.13736546242675,47.67906674331108,48.87716339478933,58.0934376365211,61.638127729580866,60.14303190817605,61.01562946938122,61.819327223152236,59.98663234529216,60.821730011304574,61.07792929526143,58.27473712981299,62.951224059650194,60.439231080338516,61.35472852164419,58.65563606525082,57.77353853059682,59.31513422204065,62.29042590649368,57.914038137918986,58.3455369319369,58.252337192417855,51.24495677711982,74.68359126923474,84.67396334746539,79.02647913143724,79.81087693914905,122.57445742059447,125.93004804216224,127.00924502594935,119.65606557711389,122.88145656257247,117.37847194268704,115.98607583425009,117.19297246113358,123.04565610365644,122.91775646111903,122.74145695385286,121.26786107235854,121.46726051506346,124.2039528663701,120.81676233311991,120.8581622174127,120.94856196475735,119.27906663077611,119.89286491529106,120.07426440830345,122.31305815117088,121.0268617459198,119.96696470819194,122.3711579887895,122.3711579887895,122.3711579887895,122.3711579887895,122.3711579887895,122.3711579887895,122.3711579887895,119.54796587923826,122.3711579887895,122.3711579887895,122.3711579887895,122.3711579887895,122.3711579887895,122.3711579887895,122.3711579887895,122.3711579887895,122.3711579887895,122.22305840270828,122.79235681159447,122.55245748208141,122.25525831271379,122.6953570826959,121.13866143345447,120.6438628163512,120.82576230796616,119.88106494827039,121.44666057263758,121.08446158493587,120.64716280712814,121.27696104692532,120.61986288342781,121.0717616204306,120.43736339348978,119.72566538259164,121.09236156285648,120.62716286302535,120.10586431998587,119.66516555168067,120.09106436134978,128.43544103992002,107.69029901984969,125.11545031885521,106.21210315121172,121.33416088705933,121.92455923697398,122.22435839907496,146.3053910957718,161.2998491882435,221.64918051983994,233.5520472529009,234.3416450460794,235.02604313327726,234.86764357598307,235.9300406067238,235.32344230208588,234.01844594937816,234.74994390493808,235.610041501079,205.1544266204967,206.00862423312734,203.3686316115577,205.4621257605183,204.22182922698315,205.06942685805978,204.04052973369127,205.61162534268672,206.0230241928813,204.29722901625067,203.9752299161956,206.15572382200338,294.6134765942775,334.99926372134104,389.42021162198546,389.4167116317675,389.7278107622866,300.599159865084,300.0552613852083,300.2035609707307,299.1663638695594,302.4997545531731,301.59455708308036,301.2780579676536,301.5354572482566,301.7607566185746,300.2461608516696,301.12785838744156,300.91585897995185,300.7396594724062,301.21915813227076,301.12935838324927,304.96934765098683,301.4288575461887,335.4485624656104,335.2124631254769,374.66645285679095,42.336981673732694,36.83029706418826,42.614580897879556,42.85378022934905,40.74378612650365,41.12408506361839,42.155582180720295,40.12658785149124,39.173690514713336,40.12658785149124,62.60232503477685,62.53552522147349,64.2098205420394,62.64942490313894,63.30192307949279,68.88030748864577,64.71301913566585,73.74569389053391,73.87329353390979,52.381353601040914,54.50604766280186,54.551347536194704,54.197748524457204,55.18854575530992,53.453350604951,54.087248833289244,53.78604967510107,54.692747141001504,52.89685216029058,54.09964879863298,51.36075645347503,54.705747104668326,54.238848409588456,53.77404970863938,53.03285178018962,52.76535252781468,53.944349232674725,54.94834642663529,55.653344456258985,60.98732954847575,70.55670280334246,68.11360963146494,69.89150466248333,69.73930508786104,70.47110304258248,87.96155415908365,86.15355921219053,84.63156346596746,84.98586247574856,84.15516479743876,85.68826051263889,86.82895732454209,84.91586267138877,84.97716250006383,112.3502859958021,138.01361427019194,120.88776213468483,152.6152734604847,144.08769729393285,130.11813633700908,120.68396270427729,129.2738386967094,128.4989408624464,132.6688292081597,92.77274071245323,92.7776406987584,112.93458436276539,112.65718513805956,113.80138194018075,106.87390130157338,115.18807806454838,183.52838706241855,175.18471038189193,185.80708069377107,184.26848499394265,185.70428098108266,185.9595802675549,185.76158082093718,183.37328749590134,240.2199286170539,238.92763222885145,441.4986660695895,441.256366746784,526.9136273460635,527.0159270601495,527.3023262597017,527.1750266154872,528.9180217440462,528.4233231266636,526.9749271747387,526.5593283362825,539.5241921012021,536.6101002457036,536.5864003119418,533.2622096026154,536.5751003435237,536.3310010257491,541.9467853303743,541.2167873706221,541.4574866978993,540.2972899404958,540.7086887906904,540.4584894899644,539.8728911266345,540.8599883678281,541.693786037474,543.4333811755354,542.1455847747561,544.666977727796,540.2968899416138,541.6870860561994,541.7216859594972,542.9001826657546,541.3942868745345,544.9921768189075,543.0118823535689,552.9300546335884,553.3151535572878,215.06179893070075,257.60108003903326,257.5357802215376,257.48328036826774,171.60472038749074,115.45057733089763,117.28527220316799,111.78798756735188,129.59273780542853,245.27481448931604,246.44421122100678,154.27646881766327,153.21637178049434,154.48946822235808,154.46416829306807,155.0972665236422,153.2551716720538,154.0188695376192,152.19477463572335,150.8751783238206,152.99817239033283,152.160274732146,153.8630699730584,152.0809749537784,152.47477385316256,150.99157799849888,151.6688761055402,149.50358215725058,154.41156844007767,153.79287016925755,154.48776822710937,159.97125290149455,162.7409451605708,162.46134594201362,164.73533958650202,158.95465574274922,158.96055572625954,164.40184051858782,164.4337404294318,163.46584313457677,154.18826906416993,155.8010645566197,153.28067160078484,158.56905682044723,152.43057397669534,153.09057213208774,154.3151687095022,154.21966897641133,140.0487085823724,154.39966847333653,149.41768239732906,149.37298252225926,147.7736869920789,146.77158979280807,144.40149641690576,147.54938761896597,150.04118065473384,147.0715889543501,148.99148358849837,147.12118881572502,144.74369546050468,146.96258925898982,150.7438786907857,149.1823830549596,149.42908236546765,150.64997895322304,148.7788841826856,145.2060941681614,144.38709645715176,145.74149267179334,157.0823609755656,156.8186617125702,154.52226813068668,154.05776942889915,155.84506443364586,156.74936190625397,158.8659559906533,160.10845251803974,156.0824637701461,156.87726154879138,158.8933559140741,155.7507646972012,159.10685531737153,156.1475635882007,154.59406793001574,158.69695646298462,164.82233934334917,223.24457606092037,217.01279347792888,210.21901246564877,212.54400596759925,212.72490546200908,208.30251782199792,209.12881551260512,209.20881528901631,210.7667109349039,209.49161449862993,210.88691059896178,211.92260770432526,210.97021036614993,207.9774187306069,212.4656061867163,209.62461412691357,210.02441300952847,212.54400596759925,209.18101536671344,210.7667109349039,209.62461412691357,261.5008691396382,261.98296779223625,284.3230053547843,342.7713419994096,342.67994225485984,341.53774544714895,342.256043439601,342.39394305418983,343.8402390119838,341.2446462663224,342.4521428915289,341.0073469295427,349.05062444964517,173.52081503225946,175.10611060156793,201.31963733822585,202.22103481893902,212.95610481583742,184.0929854844406,186.71267816274585,186.15477972199824,183.93608592295413,181.9215915531996,192.61176167558722,191.7787640037056,181.41069298109358,181.1908935954038,184.02468567532955,181.4141929713116,182.86038892938504,185.80338070411204,182.97568860713767,181.52119267226155,181.27719335420738,182.76628919238132,183.48608718064114,181.24349344839416,189.75266966637147,183.00968851211246,184.7388836792405,200.37603997545574,200.00264101905645,238.46223352957932,250.53019980120885,250.99049851473478,251.91249593787384,250.90979874028,250.90979874028,254.97388738168948,254.81478782635168,235.55364165870907,262.6943658039728,240.9622265424293,280.892014943949,78.43888077369698,78.61548028012469,78.00168199560976,78.142581601814,77.86638237375433,66.5069141219665,65.5155168927907,64.91721856495543,66.86391312420149,63.31152305266214,63.87472147859699,65.36511731313765,64.13772074354881,64.97341840788431,65.37241729273516,66.86561311945022,66.02101547998898,95.91013194385945,101.76121559085412,102.14641451427404,100.84981813808953,101.73551566268202,101.55871615681326,102.68281301511115,152.83367285008728,189.45327050315254,239.62033029285197,239.62033029285197,239.62033029285197,201.31443735275911,200.38083996204037,202.38903434940255,202.33333450507624,199.8615414134112,199.87164138518312,200.3902399357687,200.52893954812163,202.49753404616024,201.00483821804775,200.87263858752826,200.20554045197937,204.71642784464538,200.9409383966393,208.34101771439583,201.67013635862742,200.23484037008996,201.74383615264622,242.81782135628708,243.32181994767765,242.04512351587542,242.02612356897777,241.71262444516637,240.79832700050684,241.74242436187953,240.74642714556012,243.61031914136055,241.78362424673128,240.9064266983825,240.82172693510714,241.3055255829539,245.4035141296176,239.63013026546233,239.54033051644075,240.34112827831686,243.2142202484046,241.56752485070058,240.31262835797034,263.36336393421146,260.0351732360645,278.3250221183546,282.1769113528334,296.84717035139874,283.5219075937467,284.31150538692526,284.26530551604776,283.70070709402574,288.8388927334761,307.2833411836808,309.02233632341927,307.2390413074931,308.7132371873105,333.57806769339606,342.37364311092546,347.9363275639577,347.8465278149361,361.52638958153085,379.9853379912101,433.602088139481,113.18788365482736,142.99070035989425,161.76764788080806,161.31004915973597,99.8504209312726,99.48812194385037,70.58560272257101,128.7604401315905,206.58692261685974,209.1116155606767,206.31672337203094,205.5979253809763,206.65272243295794,206.57032266325442,206.42062308164498,224.59987227304657,224.62817219395203,192.34326242600713,227.23476490886998,273.5561354467625,272.51033836962705,273.10263671423155,275.9218288349621,270.77404322234247,274.2859334070736,284.3192053654048,285.8144011865301,284.5899046088362,285.3043026121882,285.05240331621343,288.2303944341484,285.29920262644197,284.7424041826201,285.491602088711,285.7152014637802,285.7152014637802,285.44600221615656,285.4042023329817,287.2860970733347,285.1092031574654,285.08680322007024,285.5125020302983,274.2719334462017,276.1450282111494,274.2343335512884,276.08292838471027,23.79463349714476,23.910533173220482,24.591031271318254,25.942127495182906,32.04171044765486,24.410231776628944,23.946733072046552,23.83523338367344,24.07653270927372,33.013007733007335,23.23623505779458,23.886933239179182,24.538231418886863,24.26343218691439,25.78932792223752,25.463028834200333,23.87413327495339,32.70290859969342,26.19212679646791,27.64772272826969,27.95722186326052,28.326620830839236,28.127521387295864,26.98112459132337,26.95372466790253,28.5663201609113,27.691122606972773,28.500020346210515,27.08192430960148,27.08192430960148,27.56812295074055,26.970824620110427,26.7299252933922,25.969027420001172,25.36572910614021,26.387826249513804,24.75793080485612,27.09272427941699,24.552531378920364,47.934166030342304,46.06737124778695,47.01536859825967,46.21587083275024,45.84177187830736,45.731172187418885,46.542569919669475,46.663369582050386,47.274067875229385,46.64786962537072,46.21587083275024,45.883771760923246,45.883771760923246,44.20047646551109,46.21587083275024,46.06217126232023,53.31435099343653,53.24465118823828,54.110048769566426,54.32724816252284,54.10964877068437,52.88115220416989,52.88115220416989,54.10964877068437,51.09685719103858,53.47155055408454,54.10964877068437,50.88525778243096,54.10964877068437,53.28575107336953,52.99745187912767,53.59385021227315,53.46095058371005,51.55095592189265,51.701455501266224,54.441147844188286,39.41108985121357,50.51355882128041,45.971271516373,46.90716890066352,47.48436728747033,48.835863510217045,53.02675179723826,47.40896749820278,54.210448488962484,53.77964969298817,66.99081276953375,66.9770128081028,69.23360650122174,69.20640657724194,56.20364291824754,44.57757541156938,45.56867264158363,82.58916917418951,36.54759785429518,33.85860536967372,34.08690473160718,36.49649799711252,37.08749634535026,34.88550249963198,37.447995337803235,34.6199032419468,35.442200943733425,36.49499800130481,36.301698541551254,37.501195189116686,36.547197855413124,37.641294797556796,34.12850461534101,33.98840500690089,37.54539506558387,36.098199110305266,35.46480088056959,34.92570238727862,41.20968482437837,38.57519218743705,38.35409280538058,38.11339347810339,39.37118996272849,40.998085415770745,40.44078697334623,38.4984924018028,40.57568659631961,41.08538517177947,42.14928219832792,41.154784977816185,40.254787493190186,44.43047582269329,41.576983797826294,42.65238079223385,44.902474503519365,44.68297511699113,45.51087280312654,40.57408660079138,44.50237562174285,47.179168140461606,47.10726834141204,46.46357014046342,47.403767512736046,46.65436960720413,48.4955644613079,42.82938029754363,43.736477762326125,47.51166721117065,49.57666143978476,45.22227360972313,47.046368511619015,46.57076984085443,45.0034742212385,46.253670727104534,47.210768052144026,46.60106975617017,46.216870829955376,45.52327276847028,694.5158589199815,86.87155720548105,87.18965631643609,90.72704642989832,87.29055603443472,87.13255647602259,98.08632586168513,96.25873096957126,98.08632586168513,98.44562485749192,98.20522552937628,97.3690278664382,98.36232509030377,97.88232643183657,96.1075313921541,96.56783010568004,101.10001743881554,100.84921813976644,97.95222623647585,98.04772596956673,97.81172662915368,97.02672882311879,96.65612985889389,97.87302645782876,98.12932574150615,101.74801562774627,106.95010108860505,106.36600272108278,105.70250457547239,101.66391586279398,104.23590867441406,105.09790626524475,105.62780478424843,101.85381533205008,102.49671353523458,105.06210636530072,103.11241181443927,103.21471152852511,104.96130664702262,107.03140086138293,104.09650906401755,110.15449213275568,101.59201606374442,107.99939815595845,117.1467725902561,116.87457335101699,126.31984695272583,125.52634917044725,134.86682306505742,134.13512511005646,133.4709269664025,135.4368214719872,135.7755205253681,135.02922261117214,135.49052132190323,135.15492225985824,136.93021729814328,138.5904126581167,135.65152087193076,140.42130754100756,134.960522803179,133.27512751363608,139.32301061060227,134.98582273246907,149.79088135428728,169.57412606273346,169.92862507195557,160.1545523891967,161.54194851160793,162.3450462670559,161.6882481027199,161.37434898002647,162.36414621367405,160.76615067986035,165.49703745765714,161.55624847164145,150.685378854285,149.36978253120284,152.4474739294622,156.3168631150309,154.31406871257653,153.18967185511713,152.44737392974167,154.28396879670183,152.2216745605416,153.97566965835716,156.20736342106807,149.51278213153788,156.34646303230306,155.06616661056233,154.13356921704877,156.33416306667985,156.24616331262754,153.28007160246176,215.4090979600459,215.51239767133683,210.27081232087497,214.8378995564699,219.17848742510057,219.04848778843237,217.10749321325562,214.53060041533038,214.91079935272458,215.72389708022396,210.89691057101317,219.39908680855447,215.5364976039807,219.51678647959946,217.06109334293714,216.26919555618676,214.53590040051762,214.0252018278526,216.0722961064947,200.83783868478938,219.31668703885092,229.96465727918164,230.33735623753734,230.97455445665253,241.02262637361974,241.2372257738428,239.97192931017918,244.61191634202876,273.0481368665514,294.13177794056156,289.60419059456973,155.05126665220575,155.22626616310524,133.80002604661405,139.4682102047886,136.44081866594777,135.85552030177934,135.92282011368525,280.25631672064145,281.2485139475814,278.8974205185768,283.08880880420054,282.4650105476342,283.256408335782,280.0715172371316,282.8162095660794,279.73811816893794,282.49981045037305,281.1056143469669,281.95261197972053,279.8687178039292,280.61801570974063,279.80271798839,289.70139032290933,293.04788096991035,293.6001794263091,295.88137305067454,293.9426784690696,295.5778738989145,292.8893814128956,294.10297802105356,398.0569874833386,402.93227385755773,384.7258247421763,403.2087730847789,302.29095513673985,302.30205510571693,310.5173321451036,310.9081310528723,310.62433184605356,311.0748305869691,319.609806732839,318.2298105897458,320.8521032607844,320.53630414340125,318.6898093041102,327.11668575210444,314.1363220305052,327.96838337172215,317.2510133253548,319.13850805005654,312.6495261859031,312.6495261859031,315.28531881921106,317.2510133253548,55.90034376592857,54.726247047373704,59.157334663069555,53.77564970416762,54.584247444243815,58.36873686709615,57.41203954093871,53.83394954122727,55.168345811766095,59.59313344506957,53.293451051849104,58.47493657028202,54.51314764295836,54.79944684278995,57.38453961779736,57.02044063540588,55.70924430002632,54.0348489797399,53.83394954122727,59.940532474135196,60.35773130811961,59.972532384699676,65.31061746545751,64.29742029720967,68.15800950737315,69.21390655628048,68.05570979328732,63.381322857580905,57.61733896715395,62.4519254551238,63.42722272929684,59.87083266893694,71.90119904565319,71.84249920971148,69.92360457276833,67.22351211916981,75.31688949924991,75.03399028991578,74.63609140199058,73.44809472228427,73.35619497913189,74.69789122926825,75.1010901023807,74.25959245425537,75.09719011328063,74.13059281479232,74.52829170327648,72.51289733603734,73.9326933678951,74.72629114989421,74.63929139304703,81.53797211214635,77.91028225105997,78.13058163535231,87.21695624013641,84.94066260207622,83.17646753276824,87.95245418451688,83.1561675895039,84.38016416859524,147.0866889121477,141.13270555274414,140.68640680009014,135.25992196639794,143.3002994946056,140.68340680847473,142.84990075341054,140.21700811199742,138.01641426236634,137.91621454241132,142.41210197700025,141.9273033319484,141.30030508432563,140.4382074937744,140.74610663323702,142.71470113127563,140.99980592418106,133.79182606953194,141.22440529645547,139.72120949768902,128.50144085545926,128.0222421947562,128.5975405868732,128.43074105305587,122.5203575717964,123.20145566821726,124.18255292618011,111.89278727445054,111.88418729848632,102.13171455535847,111.78958756288009,112.10458668249919,113.35388319088058,113.35388319088058,98.95152344357226,134.55532393565633,135.85082031491518,136.12941953626716,134.75302338311246,134.12942512598715,134.97292276852275,137.2431164236316,135.57992107204274,135.7468206055806,135.6588208515283,136.63991810949113,134.2674247402965,135.42102151614597,143.8416979814684,136.82621758880873,134.82412318439793,136.32051900216942,135.83232036662008,135.82922037528417,136.80661764358797,147.63618737637213,148.10988605244694,148.04878622321291,148.4625850666998,147.6516873330518,148.31318548425193,147.9086866147728,102.13151455591745,104.2704085779914,88.0662538664618,89.37185021749261,102.27211416296014,102.8347125905719,70.1727038765687,69.28900634638649,64.0213210688705,61.436428293304125,59.441633868490854,62.58992506943311,62.1855261996745,70.66230250820526,61.61212780224723,61.65742767564006,61.52072805769743,63.905721391956334,60.636630528633155,73.21629537013281,77.1647843346281,75.18968985475608,76.32998666777722,76.42418640450141,81.69077168509173,78.7018800386488,73.89639346934854,82.59886914707938,82.98256807469159,81.88957112947358,85.15176201208128,82.13457044473287,83.54496650286232,79.49467782288379,82.57966920074068,81.80637136200593,85.2976616043112,79.67867730862955,90.11334814510391,85.33486150034243,83.59706635725013,83.26496728542313,82.22357019599033,84.01946517670126,82.4732694981138,81.84377125747817,86.25355893270454,87.21315625075687,88.4126528983223,90.12814810373997,90.33424752771933,89.32645034437924,88.79305183515757,90.52914698300111,89.20825067473169,90.00094845924616,90.87674601150778,89.25025055734757,89.25805053554767,91.47694433403281,91.08914541787951,93.9404374488952,92.89774036309572,94.17583679098516,94.17583679098516,94.17583679098516,93.3526390917139,94.06953708807877,92.54504134884284,92.30074203162714,92.91534031390619,93.32173917807508,92.9682401660581,93.61023837175797,93.78533788237799,98.90812356486917,97.49202752267043,103.05191198352831,97.76532675883519,96.99072892373374,101.64381591897067,99.97822057408949,96.5769300802468,95.28633368729312,99.30892244468929,104.99580655059997,96.97362897152584,98.70812412384116,96.0690314997562,98.56632452015234,100.30631965709593,103.24361144775365,96.55963012859789,96.68612977504809,102.89731241561367,106.6297019840782,106.42790254808094,106.16300328843936,109.06959516489927,113.95088152234915,103.39151103439384,112.88668449663919,110.98648980743216,115.86727616627947,114.87157894912157,113.60898247791181,106.0539035933586,120.2160640119923,165.80783658901464,150.13108040347592,170.09552460549344,170.74562278855495,171.23122143137095,171.23522142019152,157.18656068434117,174.58421206020537,172.6553174512108,171.5352205817335,185.80758069237365,172.08851903533747,171.88571960213508,172.26411854456003,172.01931922874178,222.08847929205797,43.73177777546197,43.459178537340804,46.435170219837445,46.405070303962724,44.892674530909,45.86627180983329,46.374370389764934,46.47637010468921,43.965277122862155,45.21707362425641,43.75457771173916,44.138976637394975,45.666372368525806,44.94517438417885,43.567978233260035,43.611478111683624,46.54876990234135,43.378278763444975,46.49897004152537,45.267573483115974,51.36855643167512,51.36855643167512,51.415756299757724,51.797255233518634,51.650655643245116,52.64645286012353,57.10854038917872,59.401733980005766,54.81114681009009,56.69334154960458,55.275045513554524,55.871243847259,55.179645780184174,55.767144138203925,53.62115013597348,51.26385672429696,48.481164501553884,49.835760715636525]

In [22]:
len(distance)

4033

Now add distance to the dataframe.

In [23]:
index = [i for i in range(4033)]

dis_series = pd.Series(distance, index = index)

In [24]:
dis_series

0        2.640393
1        2.127094
2        2.317094
3        1.993894
4        1.863595
          ...    
4028    55.767144
4029    53.621150
4030    51.263857
4031    48.481165
4032    49.835761
Length: 4033, dtype: float64

In [25]:
gpd_domain['distance'] = dis_series

In [26]:
gpd_domain

,index,name,Postcode,Prices,Bedrooms,Bathrooms,Parkings,geometry,SA2_code,Longitude,Latitude,popu2022,popu2025,Mean Income2022,Mean Income2025,hos_count,stop_count,school_count,pct,distance
0,0,811v/162 Albert Street East Melbourne VIC 3002,3002,600.0,2,1,1,POINT (144.98553 -37.81004),206041119,-37.810043,144.985531,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000,2.640393
1,1,PO9P/191 Powlett Street East Melbourne VIC 3002,3002,675.0,2,2,1,POINT (144.98604 -37.81011),206041119,-37.810110,144.986036,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000,2.127094
2,2,203/33 Cliveden Close East Melbourne VIC 3002,3002,380.0,1,1,1,POINT (144.97672 -37.81710),206041119,-37.817100,144.976716,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000,2.317094
3,3,9/322 Albert Street East Melbourne VIC 3002,3002,990.0,3,2,1,POINT (144.98027 -37.80984),206041119,-37.809840,144.980274,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000,1.993894
4,4,107K/211 Powlett Street East Melbourne VIC 3002,3002,550.0,1,1,1,POINT (144.98599 -37.80970),206041119,-37.809699,144.985994,4965.669440,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000,1.863595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4028,4031,20 Knightsford Avenue Clyde VIC 3978,3978,500.0,4,2,2,POINT (145.34198 -38.12333),212031303,-38.123332,145.341980,21129.675792,34771.277186,71382.469390,79738.921375,0.0,0.0,5.0,0.833333,55.767144
4029,4032,6 Cecil Lane Clyde VIC 3978,3978,700.0,4,2,2,POINT (145.32807 -38.12609),212031303,-38.126093,145.328066,21129.675792,34771.277186,71382.469390,79738.921375,0.0,0.0,5.0,0.833333,53.621150
4030,4033,43 Bellman Avenue Clyde VIC 3978,3978,450.0,3,2,2,POINT (145.34012 -38.12376),212031303,-38.123765,145.340118,21129.675792,34771.277186,71382.469390,79738.921375,0.0,0.0,5.0,0.833333,51.263857
4031,4034,24 Swanston Street Clyde North VIC 3978,3978,570.0,4,2,2,POINT (145.33160 -38.12570),212031303,-38.125702,145.331599,21129.675792,34771.277186,71382.469390,79738.921375,0.0,0.0,5.0,0.833333,48.481165


In [27]:
# calculating numerical attributes according to suburb weighting
pct_col = ['popu2022', 'popu2025', 'hos_count', 'stop_count', 'school_count']
for i in pct_col:
    gpd_domain[i] = gpd_domain[i] * gpd_domain['pct']

# some school counts become not an integer, so convert to integer by ceiling, eg. from 1.6 to 2
gpd_domain['hos_count'] = gpd_domain['hos_count'].apply(np.ceil)
gpd_domain['stop_count'] = gpd_domain['stop_count'].apply(np.ceil)
gpd_domain['school_count'] = gpd_domain['school_count'].apply(np.ceil)

In [28]:
gpd_domain

,index,name,Postcode,Prices,Bedrooms,Bathrooms,Parkings,geometry,SA2_code,Longitude,Latitude,popu2022,popu2025,Mean Income2022,Mean Income2025,hos_count,stop_count,school_count,pct,distance
0,0,811v/162 Albert Street East Melbourne VIC 3002,3002,600.0,2,1,1,POINT (144.98553 -37.81004),206041119,-37.810043,144.985531,4965.66944,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000,2.640393
1,1,PO9P/191 Powlett Street East Melbourne VIC 3002,3002,675.0,2,2,1,POINT (144.98604 -37.81011),206041119,-37.810110,144.986036,4965.66944,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000,2.127094
2,2,203/33 Cliveden Close East Melbourne VIC 3002,3002,380.0,1,1,1,POINT (144.97672 -37.81710),206041119,-37.817100,144.976716,4965.66944,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000,2.317094
3,3,9/322 Albert Street East Melbourne VIC 3002,3002,990.0,3,2,1,POINT (144.98027 -37.80984),206041119,-37.809840,144.980274,4965.66944,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000,1.993894
4,4,107K/211 Powlett Street East Melbourne VIC 3002,3002,550.0,1,1,1,POINT (144.98599 -37.80970),206041119,-37.809699,144.985994,4965.66944,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.000000,1.863595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4028,4031,20 Knightsford Avenue Clyde VIC 3978,3978,500.0,4,2,2,POINT (145.34198 -38.12333),212031303,-38.123332,145.341980,17608.06316,28976.064322,71382.469390,79738.921375,0.0,0.0,5.0,0.833333,55.767144
4029,4032,6 Cecil Lane Clyde VIC 3978,3978,700.0,4,2,2,POINT (145.32807 -38.12609),212031303,-38.126093,145.328066,17608.06316,28976.064322,71382.469390,79738.921375,0.0,0.0,5.0,0.833333,53.621150
4030,4033,43 Bellman Avenue Clyde VIC 3978,3978,450.0,3,2,2,POINT (145.34012 -38.12376),212031303,-38.123765,145.340118,17608.06316,28976.064322,71382.469390,79738.921375,0.0,0.0,5.0,0.833333,51.263857
4031,4034,24 Swanston Street Clyde North VIC 3978,3978,570.0,4,2,2,POINT (145.33160 -38.12570),212031303,-38.125702,145.331599,17608.06316,28976.064322,71382.469390,79738.921375,0.0,0.0,5.0,0.833333,48.481165


In [33]:
# Delete the pct column.

gpd_domain = gpd_domain.drop(['pct'], axis=1)

In [34]:
gpd_domain

,index,name,Postcode,Prices,Bedrooms,Bathrooms,Parkings,geometry,SA2_code,Longitude,Latitude,popu2022,popu2025,Mean Income2022,Mean Income2025,hos_count,stop_count,school_count,distance
0,0,811v/162 Albert Street East Melbourne VIC 3002,3002,600.0,2,1,1,POINT (144.98553 -37.81004),206041119,-37.810043,144.985531,4965.66944,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,2.640393
1,1,PO9P/191 Powlett Street East Melbourne VIC 3002,3002,675.0,2,2,1,POINT (144.98604 -37.81011),206041119,-37.810110,144.986036,4965.66944,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,2.127094
2,2,203/33 Cliveden Close East Melbourne VIC 3002,3002,380.0,1,1,1,POINT (144.97672 -37.81710),206041119,-37.817100,144.976716,4965.66944,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,2.317094
3,3,9/322 Albert Street East Melbourne VIC 3002,3002,990.0,3,2,1,POINT (144.98027 -37.80984),206041119,-37.809840,144.980274,4965.66944,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.993894
4,4,107K/211 Powlett Street East Melbourne VIC 3002,3002,550.0,1,1,1,POINT (144.98599 -37.80970),206041119,-37.809699,144.985994,4965.66944,4555.306297,98664.943495,102957.654101,6.0,1.0,1.0,1.863595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4028,4031,20 Knightsford Avenue Clyde VIC 3978,3978,500.0,4,2,2,POINT (145.34198 -38.12333),212031303,-38.123332,145.341980,17608.06316,28976.064322,71382.469390,79738.921375,0.0,0.0,5.0,55.767144
4029,4032,6 Cecil Lane Clyde VIC 3978,3978,700.0,4,2,2,POINT (145.32807 -38.12609),212031303,-38.126093,145.328066,17608.06316,28976.064322,71382.469390,79738.921375,0.0,0.0,5.0,53.621150
4030,4033,43 Bellman Avenue Clyde VIC 3978,3978,450.0,3,2,2,POINT (145.34012 -38.12376),212031303,-38.123765,145.340118,17608.06316,28976.064322,71382.469390,79738.921375,0.0,0.0,5.0,51.263857
4031,4034,24 Swanston Street Clyde North VIC 3978,3978,570.0,4,2,2,POINT (145.33160 -38.12570),212031303,-38.125702,145.331599,17608.06316,28976.064322,71382.469390,79738.921375,0.0,0.0,5.0,48.481165


In [31]:
gpd_domain.to_csv('../data/curated/final_df.csv')